# Propensity analysis

NOTE: What is referred to as a dict is sometimes a df - need to fix this labelling at some point.

## Imports

In [ ]:
import copy
import itertools
import pickle
import random
import scipy
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns  # Note will call statsmodels for kde if installed on system, otherwise will use scipy
from collections import OrderedDict
from sklearn.cluster import KMeans

In [ ]:
sns.set()

## Functions and variable definitions

In [ ]:
"""
Defines dictionary of amino acid abbreviations
"""
aa_dict = OrderedDict({'A': 'Ala',
                       'R': 'Arg',
                       'N': 'Asn',
                       'D': 'Asp',
                       'C': 'Cys',
                       'Q': 'Gln',
                       'E': 'Glu',
                       'G': 'Gly', 
                       'H': 'His',
                       'I': 'Ile',
                       'L': 'Leu',
                       'K': 'Lys',
                       'M': 'Met',
                       'F': 'Phe',
                       'P': 'Pro',
                       'S': 'Ser',
                       'T': 'Thr',
                       'W': 'Trp',
                       'Y': 'Tyr',
                       'V': 'Val'})

In [ ]:
def define_dict(dict_vals):
    """
    Defines a dictionary of amino acid properties. Allows quick definition of new property dictionaries (can
    simply copy and paste the new property values from e.g. the Amino Acid Index) 
    Input: a list (dict_vals) of property values for the 20 (alphabetically ordered) amino acids
    Returns: a dictionary of these property values
    """
    aa_dict = OrderedDict({'A': dict_vals[0],
                           'R': dict_vals[1],
                           'N': dict_vals[2],
                           'D': dict_vals[3],
                           'C': dict_vals[4],
                           'Q': dict_vals[5],
                           'E': dict_vals[6],
                           'G': dict_vals[7],
                           'H': dict_vals[8],
                           'I': dict_vals[9],
                           'L': dict_vals[10],
                           'K': dict_vals[11],
                           'M': dict_vals[12],
                           'F': dict_vals[13],
                           'P': dict_vals[14],
                           'S': dict_vals[15],
                           'T': dict_vals[16],
                           'W': dict_vals[17],
                           'Y': dict_vals[18],
                           'V': dict_vals[19]})
    return aa_dict

In [ ]:
def remove_nan(prop_list):
    """
    Removes all instances of '', 'NaN' and np.nan from an input list
    Input: a list (prop_list) to be filtered
    Returns: the filtered list
    """
    prop_remove_list = ['', 'NaN', 'nan', np.nan]
    for prop_remove in prop_remove_list:
        if prop_remove in prop_list:
            prop_list = [x for x in prop_list if x != prop_remove]
    return prop_list

In [ ]:
def calc_distribution(df, aa_dict):
    """
    Counts the number of each amino acid in dataframe of barrel / sandwich properties
    Input: dataframe of barrel / sandwich properties and dictionary of amino acid abbreviations
    Returns: dataframe of count data
    """
    distribution_list = ['']*len(aa_dict.keys())
    fasta_list = [x for x in df['fasta_seq'].tolist() if x in list(aa_dict.keys())]
    for index, aa in enumerate(list(aa_dict.keys())):
        count = fasta_list.count(aa)
        distribution_list[index] = count / len(fasta_list)

    df_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_dict['Normalised frequency'] = distribution_list

    distribution_df = pd.DataFrame(df_dict)
    return distribution_df

In [ ]:
def calc_indv_property_propensities(df, prop, aa_dict):
    """
    Calculates propensities of the amino acids listed in the input dictionary for a (categorical) feature of
    interest (e.g. 'interior' or 'exterior')
    Input: dataframe of barrel / sandwich properties, name of the feature of interest (specified via the name of
           the corresponding column in the input dataframe), and dictionary of amino acid abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    prop_list = list(set(df[prop].tolist()))
    prop_list = remove_nan(prop_list)

    temp_lists = {}
    for prop_val in prop_list:
        temp_lists['{}_propensity_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))

        temp_lists['total_{}_count'.format(prop_val)] = df[prop].tolist().count(prop_val)

    for index, aa in enumerate(list(aa_dict.keys())):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        for prop_val in prop_list:
            temp_lists['{}_{}_count'.format(prop_val, aa)] = aa_df[prop].tolist().count(prop_val)

            if(
                min([temp_lists['{}_{}_count'.format(prop_val, aa)], temp_lists['total_{}_count'.format(prop_val)],
                     aa_df.shape[0], df.shape[0]]) > 0
            ):
                temp_lists['{}_frequency_list'.format(prop_val)][index] = copy.deepcopy(temp_lists['{}_{}_count'.format(prop_val, aa)])

                temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)] = (temp_lists['{}_{}_count'.format(prop_val, aa)]
                                                                             / temp_lists['total_{}_count'.format(prop_val)])
                temp_lists['{}_normed_frequencies_list'.format(prop_val)][index] = temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)]

                temp_lists['{}_{}_propensity'.format(prop_val, aa)] = ((temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)])
                                                                       / (aa_df.shape[0] / df.shape[0]))
                temp_lists['{}_propensity_list'.format(prop_val)][index] = temp_lists['{}_{}_propensity'.format(prop_val, aa)]
            else:
                temp_lists['{}_frequency_list'.format(prop_val)][index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(prop_val)][index] = np.nan
                temp_lists['{}_propensity_list'.format(prop_val)][index] = np.nan

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    for prop_val in prop_list:
        df_propensity_dict[prop_val] = temp_lists['{}_propensity_list'.format(prop_val)]
        df_frequency_dict[prop_val] = temp_lists['{}_frequency_list'.format(prop_val)]
        df_normed_frequencies_dict[prop_val] = temp_lists['{}_normed_frequencies_list'.format(prop_val)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)
    
    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
def calc_combined_property_propensities(df, props, aa_dict):
    """
    Calculates propensities of the amino acids listed in the input dictionary for two or more (categorical)
    features of interest (e.g. 'interior' and 'transmembrane', 'interior' and 'external', 'exterior' and
    'transmembrane', or 'exterior' and 'external')
    Input: dataframe of barrel / sandwich properties, list of names of the features of interest (specified via
           the names of the corresponding columns in the input dataframe), and dictionary of amino acid
           abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    for prop in props:
        df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
        df = df.reset_index(drop=True)

    amalgamate_list = []
    for prop in props:
        filtered_list = list(set(df[prop].tolist()))
        amalgamate_list.append(filtered_list)
    combinations = list(itertools.product(*amalgamate_list))
    combinations = ['_'.join(tup) for tup in combinations]

    for prop in props:
        df[prop] = [str(x) for x in df[prop].tolist()]
    df_props = pd.DataFrame({'combination': df[props].apply('_'.join, axis=1),
                             'FASTA': df['fasta_seq']})

    temp_lists = {}
    for combination in combinations:
        temp_lists['{}_propensity_list'.format(combination)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(combination)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(combination)] = ['']*len(list(aa_dict.keys()))

        temp_lists['total_{}_count'.format(combination)] = df_props['combination'].tolist().count(combination)

        for index, aa in enumerate(list(aa_dict.keys())):
            aa_df = df_props[df_props['FASTA'] == aa]
            aa_df = aa_df.reset_index(drop=True)
            
            temp_lists['{}_{}_count'.format(combination, aa)] = aa_df['combination'].tolist().count(combination)
            if (
                min([temp_lists['{}_{}_count'.format(combination, aa)],
                     temp_lists['total_{}_count'.format(combination)], aa_df.shape[0], df.shape[0]]) > 0
            ):
                temp_lists['{}_frequency_list'.format(combination)][index] = copy.deepcopy(temp_lists['{}_{}_count'.format(combination, aa)])

                temp_lists['{}_{}_normed_frequencies'.format(combination, aa)] = (temp_lists['{}_{}_count'.format(combination, aa)]
                                                                                / temp_lists['total_{}_count'.format(combination)])
                temp_lists['{}_normed_frequencies_list'.format(combination)][index] = temp_lists['{}_{}_normed_frequencies'.format(combination, aa)]

                temp_lists['{}_{}_propensity'.format(combination, aa)] = ((temp_lists['{}_{}_normed_frequencies'.format(combination, aa)])
                                                                          / (aa_df.shape[0] / df_props.shape[0]))
                temp_lists['{}_propensity_list'.format(combination)][index] = temp_lists['{}_{}_propensity'.format(combination, aa)]
            else:
                temp_lists['{}_frequency_list'.format(combination)][index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(combination)][index] = np.nan
                temp_lists['{}_propensity_list'.format(combination)][index] = np.nan

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    for combination in combinations:
        df_propensity_dict[combination] = temp_lists['{}_propensity_list'.format(combination)]
        df_frequency_dict[combination] = temp_lists['{}_frequency_list'.format(combination)]
        df_normed_frequencies_dict[combination] = temp_lists['{}_normed_frequencies_list'.format(combination)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
def gen_neighbouring_pairs_list(df, interaction, aa_dict, *args):
    """
    Makes a list of the amino acids in the input dataframe that form pairwise interactions at a particular
    location (e.g. HB vs. NHB positions) or via a particular type of interaction (e.g. cation-pi, hydrogen
    bonds, van der Waals interactions, etc.)
    Input: dataframe of barrel / sandwich properties, name of the interaction type / location of interest
           (specified via the name of the corresponding column in the input dataframe), and dictionary of amino
           acid abbreviations
    Returns: list of interacting amino acid pairs
    """
    if args:
        props = list(args)

    # Makes list of amino acid pairs (of which both amino acids are in the dataframe)
    domain_res_ids_list = []
    for row in range(df.shape[0]):
        domain_res_id = df['domain_ids'][row] + df['res_ids'][row]
        domain_res_ids_list.append(domain_res_id)

    neighbouring_pairs_list = []
    repeat_pairs_list = []
    if args:
        props_dict = OrderedDict()
        for prop in props:
            props_dict[prop] = OrderedDict()

    for row in range(df.shape[0]):
        domain_res_id_1 = df['domain_ids'][row] + df['res_ids'][row]
        aa_1 = df['fasta_seq'][row]
        if args:
            prop_val_1_list = ['']*len(props)
            for index, prop in enumerate(props):
                prop_val_1_list[index] = df[prop][row]

        interaction_list = df[interaction][row]
        if type(interaction_list) != list:
            interaction_list = [interaction_list]
        for res_id_2 in interaction_list:
            domain_res_id_2 = df['domain_ids'][row] + res_id_2

            # Only considers amino acid pairs in which both (canonical) amino acids are located in the domain
            if domain_res_id_2 in domain_res_ids_list:
                res_id_2_index = domain_res_ids_list.index(domain_res_id_2)
                aa_2 = df['fasta_seq'][res_id_2_index]
                if args:
                    prop_val_2_list = ['']*len(props)
                    for index, prop in enumerate(props):
                        prop_val_2_list[index] = df[prop][res_id_2_index]

                if (
                        aa_1 in list(aa_dict.keys())
                    and aa_2 in list(aa_dict.keys())
                    and (not [domain_res_id_1, domain_res_id_2] in repeat_pairs_list)
                    and (not [domain_res_id_2, domain_res_id_1] in repeat_pairs_list)
                ):  # Each amino acid pair is counted once.
                    repeat_pairs_list.append([domain_res_id_1, domain_res_id_2])
                    repeat_pairs_list.append([domain_res_id_2, domain_res_id_1])

                    neighbouring_pairs_list.append('{}_{}'.format(aa_1, aa_2))
                    
                    if args:
                        for index, prop in enumerate(props):
                            prop_val_1 = prop_val_1_list[index]
                            prop_val_2 = prop_val_2_list[index]
                            # Must include domain id in key to ensure it is unique
                            props_dict[prop]['{}_{}_{}_{}'.format(domain_res_id_1, domain_res_id_2, aa_1, aa_2)] = [
                                prop_val_1, prop_val_2
                            ]
                            props_dict[prop]['{}_{}_{}_{}'.format(domain_res_id_2, domain_res_id_1, aa_2, aa_1)] = [
                                prop_val_2, prop_val_1
                            ]
    if args:
        return neighbouring_pairs_list, props_dict
    else:
        return neighbouring_pairs_list


def calc_aa_pair_propensities(neighbouring_pairs_list, aa_dict):
    """
    Calculates propensities of amino acids at a particular location to interact with one another
    Input: list of interacting amino acid pairs output from gen_neighbouring_pairs_list, and dictionary of amino
           acid abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    # Each amino acid pair is counted in both orientations. (Note this is because positions 1 and 2 must be
    # treated independently (position 1 = object, position 2 = property of object) in the propensity calculations,
    # whereas these pairs do not have such an associated order and so must be listed in both orientations. See
    # lab notes for further details.)
    neighbouring_pairs_list = [['{}_{}'.format(aa_pair[0:1], aa_pair[-1:]), '{}_{}'.format(aa_pair[-1:], aa_pair[0:1])]
                               for aa_pair in neighbouring_pairs_list]
    neighbouring_pairs_list = [aa_pair for aa_pair_list in neighbouring_pairs_list for aa_pair in aa_pair_list]

    # Calculates propensity values
    all_pairs_count = len(neighbouring_pairs_list)
    temp_lists = {}

    for aa_1 in list(aa_dict.keys()):
        temp_lists['{}_propensity_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))

        # Calculates propensity for aa_1 to interact with aa_2 relative to all other amino acid possibilities
        # (= (aa_1_aa_2_count / aa_1_count) / (aa_2_count / all_aa_count)). 
        for aa_2_index, aa_2 in enumerate(list(aa_dict.keys())):
            aa_1_count = 0
            aa_2_count = 0
            aa_1_aa_2_count = 0

            for aa_pair in neighbouring_pairs_list:
                if aa_1 == aa_pair[0:1]:
                    aa_1_count += 1
                if aa_2 == aa_pair[-1:]:
                    aa_2_count += 1
                if aa_1 == aa_pair[0:1] and aa_2 == aa_pair[-1:]:
                    aa_1_aa_2_count += 1

            if min([all_pairs_count, aa_1_count, aa_2_count, aa_1_aa_2_count]) > 0:
                aa_1_aa_2_frequency = copy.deepcopy(aa_1_aa_2_count)
                temp_lists['{}_frequency_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_frequency

                aa_1_aa_2_normed_frequencies = aa_1_aa_2_count / aa_1_count
                temp_lists['{}_normed_frequencies_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_normed_frequencies

                aa_1_aa_2_propensity = ((aa_1_aa_2_count / aa_1_count)
                                        / (aa_2_count / all_pairs_count))
                temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_propensity
            else:
                temp_lists['{}_frequency_list'.format(aa_1)][aa_2_index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(aa_1)][aa_2_index] = np.nan
                temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index] = np.nan

    # The propensity for aa_1 to interact with aa_2 should equal the propensity for aa_2 to interact with aa_1
    for aa_1_index, aa_1 in enumerate(list(aa_dict.keys())):
        for aa_2_index, aa_2 in enumerate(list(aa_dict.keys())):
            propensity_12 = temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index]
            propensity_21 = temp_lists['{}_propensity_list'.format(aa_2)][aa_1_index]

            if not np.isnan(propensity_12) and not np.isnan(propensity_21):
                if propensity_12 - propensity_21 > 0.0001:
                    sys.exit('Error with propensity calculation: {}{} ({}) != {}{} ({})'.format(
                        aa_1, aa_2, propensity_12, aa_2, aa_1, propensity_21
                    ))

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    # FASTA label on heatmap refers to aa_2 (although propensity and frequency heatmaps will be symmetric about
    # their central axes)
    for aa_1 in list(aa_dict.keys()):
        df_propensity_dict[aa_1] = temp_lists['{}_propensity_list'.format(aa_1)]
        df_frequency_dict[aa_1] = temp_lists['{}_frequency_list'.format(aa_1)]
        df_normed_frequencies_dict[aa_1] = temp_lists['{}_normed_frequencies_list'.format(aa_1)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
def calc_voronoi_points(df, cont_prop_1, cont_prop_2, min_cluster_size=25):
    """
    Calculates cluster centres for discretisation of 2D property space.
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the correspoing column in the input dataframe), second (continuous) property of interest
           (specified via the name of the correspoing column in the input dataframe), and the minimum number of
           data points to be included in a cluster (default 25)
    Returns: numpy array of cluster coordinates, plus dataframe of cluster properties
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    prop_array = []
    for row in range(df.shape[0]):
        prop_array.append([df[cont_prop_1][row], df[cont_prop_2][row]])
    prop_array = np.array(prop_array)

    kmeans = KMeans(n_clusters=200).fit(prop_array)

    if kmeans.n_iter_ >= 300:
        sys.exit('Clustering failed to converge')

    else:
        cluster_coords = kmeans.cluster_centers_

        cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
        cluster_sizes = cluster_sizes[1]

        smallest_cluster_size = cluster_sizes.min()
        while smallest_cluster_size < min_cluster_size:
            smallest_cluster_index = np.abs(cluster_sizes).argmin()
            smallest_cluster_coords = cluster_coords[smallest_cluster_index]

            distances = np.sqrt(np.sum(np.square(cluster_coords-smallest_cluster_coords), axis=1))
            distances[smallest_cluster_index] = 1e10
            closest_cluster_index = np.abs(distances).argmin()
            closest_cluster_size = cluster_sizes[closest_cluster_index]

            cluster_sizes[closest_cluster_index] = closest_cluster_size + smallest_cluster_size
            cluster_sizes = np.delete(cluster_sizes, smallest_cluster_index)
            cluster_coords = np.delete(cluster_coords, smallest_cluster_index, axis=0)

            smallest_cluster_size = cluster_sizes.min()

        plt.clf()
        plt.figure(figsize=(15, 15))
        plt.scatter(prop_array[:,0], prop_array[:,1], alpha=0.1)
        plt.scatter(cluster_coords[:,0], cluster_coords[:,1], color='k')
        plt.show()

        cluster_df = pd.DataFrame({'Cluster size': cluster_sizes,
                                   '{}'.format(cont_prop_1): cluster_coords[:,0],
                                   '{}'.format(cont_prop_2): cluster_coords[:,1]})

        return cluster_coords, cluster_df


def calc_discrete_2d_indv_aa_propensities(df, cont_prop_1, cont_prop_2, aa_dict, cluster_coords):
    """
    Calculates amino acid propensity values for bins in 2D property space.
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the corresponding column in the input dataframe), second (continuous) property of interest
           (specified via the name of the corresponding column in the input dataframe), dictionary of amino acid
           abbreviations, and coordinates of cluster centres
    Returns: dataframe of propensity values, plus dataframes of frequency and normalised frequency values (since
             propensity values can be skewed by very small sample sizes)
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    voronoi_class_dict = OrderedDict()

    for voronoi_index in range(cluster_coords.shape[0]):
        voronoi_class_dict[voronoi_index] = [0]*(len(aa_dict)+1)
        propensity_dict[voronoi_index] = [np.nan]*len(aa_dict)
        frequency_dict[voronoi_index] = [np.nan]*len(aa_dict)
        normed_frequencies_dict[voronoi_index] = [np.nan]*len(aa_dict)

    for row in range(df.shape[0]):
        fasta = df['fasta_seq'][row]
        aa_index = list(aa_dict.keys()).index(fasta)

        val_1 = df[cont_prop_1][row]
        val_2 = df[cont_prop_2][row]
        prop_vals = np.array([val_1, val_2])

        distances = np.sqrt(np.sum(np.square(cluster_coords-prop_vals), axis=1))
        voronoi_index = np.abs(distances).argmin()
        voronoi_class_dict[voronoi_index][aa_index] += 1
        voronoi_class_dict[voronoi_index][-1] +=1

    for voronoi_index in list(voronoi_class_dict.keys()):
        for aa_index, aa in enumerate(list(aa_dict.keys())):
            class_aa_count = voronoi_class_dict[voronoi_index][aa_index]
            class_all_aas_count = voronoi_class_dict[voronoi_index][-1]

            total_aa_count = 0
            for voronoi_class_list in list(voronoi_class_dict.values()):
                total_aa_count += voronoi_class_list[aa_index]

            try:
                frequency_dict[voronoi_index][aa_index] = copy.deepcopy(class_aa_count)
                normed_frequencies_dict[voronoi_index][aa_index] = class_aa_count / class_all_aas_count
                propensity_dict[voronoi_index][aa_index] = (  (class_aa_count / class_all_aas_count)
                                                            / (total_aa_count / df.shape[0]))
            except ZeroDivisionError:
                pass

    propensity_df = pd.DataFrame(propensity_dict)
    frequency_df = pd.DataFrame(frequency_dict)
    normed_frequencies_df = pd.DataFrame(normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
def bootstrap_discrete_propensities(df, prop, aa_dict, num_bootstrap_samples, propensity_calc_func, *args):
    """
    Calculates 95% confidence limits about propensity values calculated for discrete categorical features
    Input: dataframe of barrel / sandwich properties, name(s) of the feature(s) of interest (specified via a
           name / list of names of the corresponding column(s) in the input dataframe, dictionary of amino
           acid abbreviations, the number of bootstrap samples to collect, the name of the propensity
           calculation function (calc_indv_property_propensities, calc_combined_property_propensities,
           calc_aa_pair_propensities, or calc_discrete_2d_indv_aa_propensities), and the minimum number of data
           points in each cluster (if calculating propensities for bins in 2D property space, e.g. phi vs. psi)
    Returns: dictionary of bootstrapped propensity values, plus dictionaries of the bootstrapped frequencies
             and normalised frequencies (since extreme propensity values can result from low counts)
    """
    if propensity_calc_func == calc_indv_property_propensities:
        prop_vals = list(set(df[prop].tolist()))
        prop_vals = remove_nan(prop_vals)

    elif propensity_calc_func == calc_combined_property_propensities:
        prop_vals = []
        for sub_prop in prop:
            vals = list(set(df[sub_prop].tolist()))
            vals = remove_nan(vals)
            prop_vals.append(vals)
        prop_vals = list(itertools.product(*prop_vals))
        prop_vals = ['_'.join(tup) for tup in prop_vals]

    elif propensity_calc_func == calc_aa_pair_propensities:
        prop_vals = list(aa_dict.keys())
        neighbouring_pairs_list = gen_neighbouring_pairs_list(df, prop, aa_dict)

    elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
        cluster_coords, cluster_df = calc_voronoi_points(df, prop[0], prop[1], args)
        prop_vals = [num for num in range(cluster_coords.shape[0])]

    aa_list = list(aa_dict.keys())

    bootstrap_propensities_dict = OrderedDict({'FASTA': aa_list})
    bootstrap_frequencies_dict = OrderedDict({'FASTA': aa_list})
    bootstrap_normed_frequencies_dict = OrderedDict({'FASTA': aa_list})

    for prop_val in prop_vals:
        bootstrap_propensities_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]
        bootstrap_frequencies_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]
        bootstrap_normed_frequencies_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]

    for num_1 in range(num_bootstrap_samples):
        if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                    calc_discrete_2d_indv_aa_propensities]:
            # Samples input dataframe with replacement
            indices = []
            for num_2 in range(df.shape[0]):
                random_num = random.randint(0, (df.shape[0]-1))
                indices.append(random_num)
            new_df = df.iloc[indices]
            new_df = new_df.reset_index(drop=True)

            if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities]:
                propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                    new_df, prop, aa_dict
                )
            elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                    new_df, prop[0], prop[1], aa_dict, cluster_coords
                )

        elif propensity_calc_func == calc_aa_pair_propensities:
            # Samples neighbouring_pairs_list with replacement
            resampled_pairs_list = random.choices(neighbouring_pairs_list, k=len(neighbouring_pairs_list))
            
            propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                resampled_pairs_list, aa_dict
            )

        for prop_val in prop_vals:
            for index, num_3 in enumerate(propensity_df[prop_val].tolist()):
                bootstrap_propensities_dict[prop_val][index][num_1] = propensity_df[prop_val][index]
                bootstrap_frequencies_dict[prop_val][index][num_1] = frequency_df[prop_val][index]
                bootstrap_normed_frequencies_dict[prop_val][index][num_1] = normed_frequencies_df[prop_val][index]

    return bootstrap_propensities_dict, bootstrap_frequencies_dict, bootstrap_normed_frequencies_dict

In [ ]:
def plot_bar_graphs(df, propensity_or_frequency):
    """
    Plots a bar graph of property / propensity of interest vs. amino acid identity
    Input: dataframe of barrel / sandwich properties, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values
           (specified via the name of the corresponding column in the input dataframe)
    """
    reshaped_df = pd.melt(df, id_vars='FASTA', var_name='Property', value_name=propensity_or_frequency)
    reshaped_df = reshaped_df.dropna()
    reshaped_df = reshaped_df.reset_index(drop=True)

    plt.clf()
    sns.catplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=reshaped_df, kind='bar', height=15)
    plt.show()

In [ ]:
def gen_95_conf_intervals(bootstrap_propensity_dict, propensity_or_frequency, middle=False):
    """
    Calculates 95% confidence intervals from bootstrapped sample.
    Input: dictionary of bootstrapped propensity values, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values
    Returns: dictionary of lower percentile propensity values, dictionary of upper percentile propensity values
    """
    aa_list = bootstrap_propensity_dict['FASTA']
    
    lower_percentile_dict = OrderedDict({'FASTA': aa_list})
    upper_percentile_dict = OrderedDict({'FASTA': aa_list})
    if middle is True:
        middle_percentile_dict = OrderedDict({'FASTA': aa_list})

    for prop_val in list(bootstrap_propensity_dict.keys()):
        if prop_val != 'FASTA':
            lower_percentile_dict[prop_val] = [np.nan]*len(aa_list)
            upper_percentile_dict[prop_val] = [np.nan]*len(aa_list)
            if middle is True:
                middle_percentile_dict[prop_val] = [np.nan]*len(aa_list)

            for index, propensity_list in enumerate(bootstrap_propensity_dict[prop_val]):
                propensity_list = [val for val in propensity_list if not np.isnan(val)]
                if propensity_list:
                    lower_percentile = np.percentile(propensity_list, 2.5)
                    upper_percentile = np.percentile(propensity_list, 97.5) 
                    middle_percentile = np.percentile(propensity_list, 50)
                else:
                    lower_percentile = np.nan
                    upper_percentile = np.nan
                    middle_percentile = np.nan

                lower_percentile_dict[prop_val][index] = lower_percentile
                upper_percentile_dict[prop_val][index] = upper_percentile
                if middle is True:
                    middle_percentile_dict[prop_val][index] = middle_percentile

    if middle is False:
        return lower_percentile_dict, upper_percentile_dict
    if middle is True:
        return lower_percentile_dict, upper_percentile_dict, middle_percentile_dict


def plot_bar_graphs_with_conf_limits(bootstrap_df, lower_percentile_dict, upper_percentile_dict,
                                     propensity_or_frequency):
    """
    Plots a bar graph of property / propensity of interest vs. amino acid identity with bootstrapped 95%
    confidence limits
    Input: dataframe of initial propensity values, dataframe of bootstrapped lower percentile values, dataframe
           of bootstrapped upper percentile values, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values
    """
    reshaped_df = pd.melt(bootstrap_df, id_vars='FASTA', var_name='Property', value_name=propensity_or_frequency)
    reshaped_df = reshaped_df.dropna()
    reshaped_df = reshaped_df.reset_index(drop=True)

    lower_percentile_df = pd.DataFrame(lower_percentile_dict)
    lower_percentile_df = pd.melt(lower_percentile_df, id_vars='FASTA', var_name='Property',
                                  value_name=propensity_or_frequency)
    upper_percentile_df = pd.DataFrame(upper_percentile_dict)
    upper_percentile_df = pd.melt(upper_percentile_df, id_vars='FASTA', var_name='Property',
                                  value_name=propensity_or_frequency)

    # Plots bar graph with confidence limits
    plt.clf()
    plt.figure(figsize=(15,10))
    sns.barplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=reshaped_df)
    sns.stripplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=lower_percentile_df,
                  jitter=False, dodge=True, palette='dark')
    sns.stripplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=upper_percentile_df,
                  jitter=False, dodge=True, palette='dark')
    plt.show()

In [ ]:
def print_conf_limits(prop_list, aa_list, lower_percentile_dict, df, upper_percentile_dict):
    """
    Prints confidence limits about discrete property propensity values for each aa
    Input: list of discrete properties, list of amino acids, dictionary of lower percentile values, dataframe of
           propensity (or frequency or normalised frequency) values, dictionary of upper percentile values
    """
    for prop in prop_list:
        for index, aa in enumerate(aa_list):
            print(aa, prop)
            print('[{}, {}, {}]'.format(
                lower_percentile_dict[prop][index], df[prop][index], upper_percentile_dict[prop][index]
            ))

In [ ]:
def iterate_bootstrap_propensities(df, prop, aa_dict, bootstrap_samples_list, propensity_calc_func, propensity_df,
                                   frequency_df, normed_frequencies_df, *args):
    """
    Calculates 95% confidence limits about propensity, frequency and normalised frequency values calculated for
    discrete categorical features whilst varying the size of the bootstrap population (to determine the minimum
    number of bootstrap samples required to obtain an accurate estimate of the confidence limits)
    Input: dataframe of barrel / sandwich properties, name(s) of the feature(s) of interest (specified via a
           name / list of names of the corresponding column(s) in the input dataframe), dictionary of amino
           acid abbreviations, list of number of bootstrap samples to use in the confidence limit calculation,
           name of the propensity calculation function (calc_indv_property_propensities,
           calc_combined_property_propensities, calc_aa_pair_propensities, or calc_discrete_2d_indv_aa_propensities),
           dataframe of (non-bootstrapped) propensity values, dataframe of (non-bootstrapped) frequency values,
           dataframe of (non-bootstrapped) normalised frequency values, and minimum number of data points in each
           cluster (calc_discrete_2d_indv_aa_propensities)
    Returns: dictionaries of propensity, frequency and normalised frequency values vs. number of bootstrap samples
    """
    dfs = {'Propensity': propensity_df,
           'Frequency': frequency_df,
           'Normalised frequency': normed_frequencies_df}

    iterated_dicts = {'Propensity': OrderedDict(),
                      'Frequency': OrderedDict(),
                      'Normalised frequency': OrderedDict()}

    for num in bootstrap_samples_list:
        print(num)
        (bootstrap_propensities_dict, bootstrap_frequencies_dict, bootstrap_normed_frequencies_dict
        ) = bootstrap_discrete_propensities(df, prop, aa_dict, num, propensity_calc_func, args)
        bootstrapped_dicts = {'Propensity': bootstrap_propensities_dict,
                              'Frequency': bootstrap_frequencies_dict,
                              'Normalised frequency': bootstrap_normed_frequencies_dict}

        for output_type, iterated_dict in iterated_dicts.items():
            bootstrap_dict = bootstrapped_dicts[output_type]
            
            if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                        calc_aa_pair_propensities]:
                lower_percentile, upper_percentile = gen_95_conf_intervals(
                    bootstrap_dict, output_type
                )
            elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                lower_percentile, upper_percentile, middle_percentile = gen_95_conf_intervals(
                    bootstrap_dict, output_type, middle=True
                )
            iterated_dict[num] = OrderedDict({'FASTA': list(aa_dict.keys())})

            for prop_val in list(bootstrap_dict.keys()):
                if prop_val != 'FASTA':
                    iterated_dict[num][prop_val] = ['']*len(list(aa_dict.keys()))

                    if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                                calc_aa_pair_propensities]:
                        for index, aa in enumerate(list(aa_dict.keys())):
                            iterated_dict[num][prop_val][index] = [
                                lower_percentile[prop_val][index], dfs[output_type][prop_val][index],
                                upper_percentile[prop_val][index],
                            ]
                    elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                        for index, aa in enumerate(list(aa_dict.keys())):
                            iterated_dict[num][prop_val][index] = [
                                lower_percentile[prop_val][index], middle_percentile[prop_val][index],
                                upper_percentile[prop_val][index],
                            ]

    for output_type, iterated_dict in iterated_dicts.items():
        for index, aa in enumerate(list(aa_dict.keys())):
            print(aa)
            x_vals = []
            y_vals = []
            hue_vals = []

            for prop_val in list(iterated_dict[min(bootstrap_samples_list)].keys()):
                if prop_val != 'FASTA':
                    for num in bootstrap_samples_list:
                        percentiles = iterated_dict[num][prop_val][index]
                        x_vals += [num]*3
                        y_vals += [percentiles[0], percentiles[1], percentiles[2]]
                        hue_vals += [prop_val]*3
            
            melted_df = pd.DataFrame({'Bootstrap samples': x_vals,
                                      '{}'.format(output_type): y_vals,
                                      'Property': hue_vals})
            plt.clf()
            plt.figure(figsize=(8, 5))
            sns.swarmplot(x='Bootstrap samples', y=output_type, hue='Property', data=melted_df)
            plt.show()

    return iterated_dicts[0], iterated_dicts[1], iterated_dicts[2]

In [ ]:
def plot_heat_map(df):
    """
    Plots a heat map.
    Input: dataframe of propensity / count data of property of interest vs. amino acid identity (which must be in
           a column labelled 'FASTA')
    """
    df = df.set_index('FASTA', drop=True)  # "FASTA" label on y-axis of heat map refers to amino acid 2

    plt.clf()
    plt.figure(figsize=(20, 10))
    sns.heatmap(df)
    plt.show()

In [ ]:
def plot_aa_kdes(df, cont_props, indv_or_comp, aa_dict):
    """
    Plots kernel density estimate of the distribution of each amino acid of interest vs. 1 / 2 continuous
    property/ies of interest (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via a list
           of the name(s) of the corresponding column in the input dataframe), whether to plot the distribution of
           the amino acid alone ('individual') or whether to also plot the distribution of all amino acids
           ('comparison'), and dictionary of amino acid abbreviations
    """
    for prop in cont_props:
        df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
        df = df.reset_index(drop=True)

    for aa in list(aa_dict.keys()):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        num_prop_vals = []
        for prop in cont_props:
            num_prop_vals.append(len(df[prop]))
            num_prop_vals.append(len(aa_df[prop]))

        if min(num_prop_vals) >= 10:
            print(aa)

            plt.clf()
            plt.figure(figsize=(10, 5))
            if 'z_coords' in cont_props:
                axes = plt.gca()

            if len(cont_props) == 1:
                if indv_or_comp == 'comparison':
                    sns.distplot(df[cont_props[0]], hist=False, kde_kws={'bw':'scott'})
                sns.distplot(aa_df[cont_props[0]], hist=False, rug=True, kde_kws={'bw':'scott'},
                             rug_kws={'alpha': 0.1})
            elif len(cont_props) == 2:
                if indv_or_comp == 'comparison':
                    sns.kdeplot(data=df[cont_props[0]], data2=df[cont_props[1]], bw='scott', shade=True,
                                shade_lowest=False, cbar=True)
                sns.kdeplot(data=aa_df[cont_props[0]], data2=aa_df[cont_props[1]], bw='scott', shade=False, cbar=True)
            else:
                return('Too many properties specified')

            plt.show()

In [ ]:
def plot_1d_indv_aa_propensities(df, cont_prop, aa_dict):
    """
    Plots propensity of each amino acid of interest vs. a continuous property of interest (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via the
           name of the corresponding column in the input dataframe), and dictionary of amino acid abbreviations
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    propensities_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        if (
            min([df.shape[0], aa_df.shape[0]]) >= 10
        ):
            print(aa)

            plt.clf()
            overall_data = np.asarray(df[cont_prop].tolist())
            overall_data = overall_data.astype(np.float64)
            x_values_overall, y_values_overall = sns.distributions._statsmodels_univariate_kde(
                data=overall_data, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                cumulative=False
            )

            # Determines range over which interpolation is carried out
            min_x_vals = [min(df[cont_prop]), min(aa_df[cont_prop])]
            max_x_vals = [max(df[cont_prop]), max(aa_df[cont_prop])]
            x_range = [max(min_x_vals), min(max_x_vals)]

            plt.clf()
            indv_data = np.asarray(aa_df[cont_prop].tolist())
            indv_data = indv_data.astype(np.float64)
            x_values_indv, y_values_indv = sns.distributions._statsmodels_univariate_kde(
                data=indv_data, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                cumulative=False
            )

            x_values_indv_copy = list(copy.deepcopy(x_values_indv))
            propensities = ['']*len(x_values_indv)

            # Interpolate overall KDE since it has been calculated from more data points
            for index_1, value_1 in np.ndenumerate(x_values_indv):
                index_1 = index_1[0]
                x_indv = x_values_indv[index_1]
                y_indv = y_values_indv[index_1]

                if (   x_indv < x_range[0]
                    or x_indv > x_range[-1]
                ):
                    x_values_indv_copy[index_1] = ''

                else:
                    interpolate_x_indices = []
                    for index_2, value_2 in np.ndenumerate(x_values_overall):
                        index_2 = index_2[0]
                        if x_values_overall[index_2] == x_indv:
                            if index_2 < (len(x_values_overall)-1):
                                interpolate_x_indices = [index_2, index_2+1]
                            else:
                                interpolate_x_indices = [index_2-1, index_2]
                            break
                        elif (    x_values_overall[index_2] < x_indv
                            and x_values_overall[index_2+1] > x_indv
                        ):
                            interpolate_x_indices = [index_2, index_2+1]
                            break

                    x_1 = x_values_overall[interpolate_x_indices[0]]
                    x_2 = x_values_overall[interpolate_x_indices[1]]
                    y_1 = y_values_overall[interpolate_x_indices[0]]
                    y_2 = y_values_overall[interpolate_x_indices[1]]

                    y_1_weight = abs(x_2 - x_indv) / abs(x_2 - x_1)
                    y_2_weight = abs(x_indv - x_1) / abs(x_2 - x_1)

                    y_overall = (y_1*y_1_weight) + (y_2*y_2_weight)

                    propensity = y_indv / y_overall
                    propensities[index_1] = propensity

            x_values_indv_copy = [x for x in x_values_indv_copy if x != '']
            propensities = [y for y in propensities if y != '']

            # Updates dictionary of amino acid propensities
            propensities_dict[aa] = np.array([x_values_indv_copy,
                                              propensities])

            plt.clf()
            plt.figure(figsize=(10, 5))
            axes = plt.gca()
            plt.plot(np.array(x_values_indv_copy), np.array(propensities))
            plt.show()

    return propensities_dict

In [ ]:
def plot_2d_indv_aa_propensities(df, cont_prop_1, cont_prop_2, aa_dict, cont_prop_1_range, cont_prop_2_range):
    """
    Calculates propensity of each amino acid of interest vs. two propensities of interest (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of the first continuous property of interest (specified
           via the name of the corresponding column in the input dataframe), name of the second continuous property
           of interest (specified via the name of the corresponding column in the input dataframe), dictionary of
           amino acid abbreviations, range of values over which to calculate propensity values for continuous
           property 1, and range of values over which to calculate propensity values for continuous property 2
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)
    
    propensities_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        if min([df.shape[0], aa_df.shape[0]]) >= 10:
            print(aa)

            plt.clf()
            overall_data_1 = np.asarray(df[cont_prop_1].tolist())
            overall_data_1 = overall_data_1.astype(np.float64)
            overall_data_2 = np.asarray(df[cont_prop_2].tolist())
            overall_data_2 = overall_data_2.astype(np.float64)
            x_values_overall, y_values_overall, z_values_overall = sns.distributions._statsmodels_bivariate_kde(
                x=overall_data_1, y=overall_data_2, bw='scott', gridsize=100, cut=3,
                clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
            )

            plt.clf()
            indv_data_1 = np.asarray(aa_df[cont_prop_1].tolist())
            indv_data_1 = indv_data_1.astype(np.float64)
            indv_data_2 = np.asarray(aa_df[cont_prop_2].tolist())
            indv_data_2 = indv_data_2.astype(np.float64)
            x_values_indv, y_values_indv, z_values_indv = sns.distributions._statsmodels_bivariate_kde(
                x=indv_data_1, y=indv_data_2, bw='scott', gridsize=100, cut=3,
                clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
            )

            propensities = np.full(z_values_indv.shape, np.nan)
            # Interpolate overall KDE since it has been calculated from more data points
            for index_1, value_1 in np.ndenumerate(z_values_indv):
                column = index_1[0]
                row = index_1[1]

                x_indv = x_values_indv[column][row]
                y_indv = y_values_indv[column][row]
                z_indv = z_values_indv[column][row]

                # Finds x,y pairs in overall z distribution for interpolation
                if (    x_indv >= cont_prop_1_range[0]
                    and x_indv <= cont_prop_1_range[-1]
                    and y_indv >= cont_prop_2_range[0]
                    and y_indv <= cont_prop_2_range[-1]
                ):
                    interpolate_x_indices = []
                    interpolate_y_indices = []

                    x_distribution = x_values_overall[0]
                    for index_2, value_2 in np.ndenumerate(x_distribution):
                        index_2 = index_2[0]
                        if x_distribution[index_2] == x_indv:
                            if index_2 != (len(x_distribution)-1):
                                interpolate_x_indices = [index_2, index_2+1]
                            else:
                                interpolate_x_indices = [index_2-1, index_2]
                            break
                        elif (    x_distribution[index_2] < x_indv
                              and x_distribution[index_2+1] > x_indv
                        ):
                            interpolate_x_indices = [index_2, index_2+1]
                            break

                    y_distribution = np.transpose(y_values_overall)[0]
                    for index_3, value_3 in np.ndenumerate(y_distribution):
                        index_3 = index_3[0]
                        if y_distribution[index_3] == y_indv:
                            if index_3 != (len(y_distribution)-1):
                                interpolate_y_indices = [index_3, index_3+1]
                            else:
                                interpolate_y_indices = [index_3-1, index_3]
                            break
                        elif (    y_distribution[index_3] < y_indv
                              and y_distribution[index_3+1] > y_indv
                        ):
                            interpolate_y_indices = [index_3, index_3+1]
                            break

                    # Bilinear interpolation calculation
                    x1 = x_values_overall[0][interpolate_x_indices[0]]
                    x2 = x_values_overall[0][interpolate_x_indices[1]]
                    y1 = y_values_overall[interpolate_y_indices[0]][0]
                    y2 = y_values_overall[interpolate_y_indices[1]][0]
                    z_x1y1 = z_values_overall[interpolate_y_indices[0]][interpolate_x_indices[0]]
                    z_x2y1 = z_values_overall[interpolate_y_indices[0]][interpolate_x_indices[1]]
                    z_x1y2 = z_values_overall[interpolate_y_indices[1]][interpolate_x_indices[0]]
                    z_x2y2 = z_values_overall[interpolate_y_indices[1]][interpolate_x_indices[1]]

                    # Interpolation in x
                    x1_weight = abs(x2 - x_indv) / abs(x2 - x1)
                    x2_weight = abs(x_indv - x1) / abs(x2 - x1)
                    z_xy1 = (z_x1y1*x1_weight) + (z_x2y1*x2_weight)
                    z_xy2 = (z_x1y2*x1_weight) + (z_x2y2*x2_weight)

                    # Interpolation in y
                    y1_weight = abs(y2 - y_indv) / abs(y2 - y1)
                    y2_weight = abs(y_indv - y1) / abs(y2 - y1)
                    z_overall = (z_xy1*y1_weight) + (z_xy2*y2_weight)

                    # Propensity calculation
                    propensity = z_indv / z_overall
                    propensities[column][row] = propensity

            # Plots individual pairwise KDEs used in propensity calculation
            plt.clf()
            fig, ax = plt.subplots(figsize=(10, 5))
            contours_1 = ax.contour(x_values_overall, y_values_overall, z_values_overall, 10, cmap='Blues')
            colourbar_1 = fig.colorbar(contours_1)
            contours_2 = ax.contour(x_values_indv, y_values_indv, z_values_indv, 10, cmap='Oranges')
            colourbar_2 = fig.colorbar(contours_2)
            plt.show()

            # Plots propensity KDE
            plt.clf()
            fig, ax = plt.subplots(figsize=(8, 5))
            contours_1 = ax.contourf(x_values_indv, y_values_indv, propensities, 10, cmap='Oranges')
            colourbar_1 = fig.colorbar(contours_1)
            plt.show()
            
            # Updates dictionary of amino acid propensities
            propensities_dict[aa] = np.array([x_values_indv,
                                              y_values_indv,
                                              propensities])

    return propensities_dict

In [ ]:
def plot_1d_pairwise_aa_propensities(df, interaction, cont_prop, aa_dict):
    """
    Calculates propensities of amino acid pairs to interact with one another at different values of a continuous
    property of interest
    Input: dataframe of barrel / sandwich properties, name of interaction type of interest (specified via the
           name of the corresponding column in the input dataframe), name of continuous property of interest
           (specified via the name of the corresponding column in the input dataframe), and dictionary of amino
           acid abbreviations 
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    propensities_dict = OrderedDict()

    neighbouring_pairs_list, prop_dicts = gen_neighbouring_pairs_list(df, interaction, aa_dict, cont_prop)
    prop_dict = prop_dicts[cont_prop]
    propensity_df, frequency_df, normed_frequencies_df = calc_aa_pair_propensities(neighbouring_pairs_list, aa_dict)
    propensity_df = propensity_df.set_index('FASTA', drop=True)

    for aa_1 in list(aa_dict.keys()):
        for aa_2 in list(aa_dict.keys()):
            all_aa_list = []
            aa_1_list = []
            aa_2_list = []
            aa_1_aa_2_list = []

            for aa_pair in list(prop_dict.keys()):
                aas = aa_pair.split('_')[2:]

                all_aa_list.append(prop_dict[aa_pair][0])
                if aa_1 == aas[0]:
                    aa_1_list.append(prop_dict[aa_pair][0])
                if aa_2 == aas[-1]:
                    aa_2_list.append(prop_dict[aa_pair][0])  # Want z_coords of aas interacting with aa_2
                if aa_1 == aas[0] and aa_2 == aas[-1]:
                    aa_1_aa_2_list.append(prop_dict[aa_pair][0])

            if (
                    (not np.isnan(propensity_df[aa_1][aa_2]))
                and (min([len(all_aa_list), len(aa_1_list), len(aa_2_list), len(aa_1_aa_2_list)]) >= 10)
            ):
                print('Propensity for {} to interact with {}'.format(aa_1, aa_2))

                plt.clf()
                all_aa_array = np.asarray(all_aa_list)
                all_aa_array = all_aa_array.astype(np.float64)
                x_all_aa_list, y_all_aa_list = sns.distributions._statsmodels_univariate_kde(
                    data=all_aa_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_1_array = np.asarray(aa_1_list)
                aa_1_array = aa_1_array.astype(np.float64)
                x_aa_1_list, y_aa_1_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_1_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_2_array = np.asarray(aa_2_list)
                aa_2_array = aa_2_array.astype(np.float64)
                x_aa_2_list, y_aa_2_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_2_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_1_aa_2_array = np.asarray(aa_1_aa_2_list)
                aa_1_aa_2_array = aa_1_aa_2_array.astype(np.float64)
                x_aa_1_aa_2_list, y_aa_1_aa_2_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_1_aa_2_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                x_coord_lists = [x_all_aa_list, x_aa_1_list, x_aa_2_list]
                y_coord_lists = [y_all_aa_list, y_aa_1_list, y_aa_2_list]

                x_aa_1_aa_2_list_copy = list(copy.deepcopy(x_aa_1_aa_2_list))
                interpolated_y_coord_lists = [(['']*len(x_aa_1_aa_2_list_copy)) for num in range(3)]
                propensities = ['']*len(x_aa_1_aa_2_list_copy)

                x_min = max([min(all_aa_list), min(aa_1_list), min(aa_2_list), min(aa_1_aa_2_list)])
                x_max = min([max(all_aa_list), max(aa_1_list), max(aa_2_list), max(aa_1_aa_2_list)])

                for index_1, value_1 in np.ndenumerate(x_aa_1_aa_2_list):
                    index_1 = index_1[0]
                    x_aa_1_aa_2 = x_aa_1_aa_2_list[index_1]
                    y_aa_1_aa_2 = y_aa_1_aa_2_list[index_1]

                    if (
                           x_aa_1_aa_2 < x_min
                        or x_aa_1_aa_2 > x_max
                    ):
                        x_aa_1_aa_2_list_copy[index_1] = ''
                    else:
                        for index_2, value_2 in np.ndenumerate(x_coord_lists):
                            index_2 = index_2[0]
                            x_coord_list = x_coord_lists[index_2]
                            y_coord_list = y_coord_lists[index_2]

                            interpolate_x_indices = []
                            for index_3, value_3 in np.ndenumerate(x_coord_list):
                                index_3 = index_3[0]
                                x_coord = copy.deepcopy(value_3)
                                if x_aa_1_aa_2 == x_coord:
                                    if index_3 != (len(x_coord_list)-1):
                                        interpolate_x_indices = [index_3, index_3+1]
                                    else:
                                        interpolate_x_indices = [index_3-1, index_3]
                                    break
                                elif x_coord_list[index_3] < x_aa_1_aa_2 and x_coord_list[index_3+1] > x_aa_1_aa_2:
                                    interpolate_x_indices = [index_3, index_3+1]
                                    break

                            # Linear interpolation
                            x_1 = x_coord_list[interpolate_x_indices[0]]
                            x_2 = x_coord_list[interpolate_x_indices[1]]
                            y_1 = y_coord_list[interpolate_x_indices[0]]
                            y_2 = y_coord_list[interpolate_x_indices[1]]

                            y_1_weight = abs(x_2 - x_aa_1_aa_2) / abs(x_2 - x_1)
                            y_2_weight = abs(x_aa_1_aa_2 - x_1) / abs(x_2 - x_1)

                            y = (y_1*y_1_weight) + (y_2*y_2_weight)
                            interpolated_y_coord_lists[index_2][index_1] = y


                for index_1, value_1 in enumerate(x_aa_1_aa_2_list_copy):
                    if value_1 != '':
                        y_all_aa = interpolated_y_coord_lists[0][index_1]
                        y_aa_1 = interpolated_y_coord_lists[1][index_1]
                        y_aa_2 = interpolated_y_coord_lists[2][index_1]
                        y_aa_1_aa_2 = y_aa_1_aa_2_list[index_1]
                        y_ratio = ((y_aa_1_aa_2 / y_aa_1)
                                   / (y_aa_2 / y_all_aa))

                        propensity = y_ratio * propensity_df[aa_1][aa_2]
                        propensities[index_1] = propensity

                # Plots kernel density estimates used in the calculation of the pairwise amino acid propensity plot
                plt.clf()
                plt.figure(figsize=(10, 5))
                ax = plt.gca()
                sns.distplot(all_aa_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.1})
                sns.distplot(aa_1_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.4})
                sns.distplot(aa_2_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.4})
                sns.distplot(aa_1_aa_2_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.8})
                plt.show()

                # Plots pairwise amino acid propensities vs. z-coordinate
                x_aa_1_aa_2_list_copy = [x for x in x_aa_1_aa_2_list_copy if x != '']
                propensities = [y for y in propensities if y != '']

                plt.clf()
                plt.figure(figsize=(10, 5))
                ax = plt.gca()
                plt.plot(np.array(x_aa_1_aa_2_list_copy), np.array(propensities))
                plt.show()

                # Updates dictionary of amino acid propensities
                propensities_dict['{}_{}'.format(aa_1, aa_2)] = np.array([x_aa_1_aa_2_list_copy,
                                                                          propensities])
                
    return propensities_dict

In [ ]:
def plot_2d_pairwise_aa_propensities(df, interaction, prop_1, prop_2, aa_dict, cont_prop_1_range, cont_prop_2_range):
    """
    Calculates propensities of the different amino acids to interact with one another vs. two continuous properties
    of interest (e.g. phi and psi).
    Input: dataframe of barrel / sandwich properties, name of continuous property 1 (specified by the name of its
           corresponding column in the input dataframe), name of continuous property 2 (specified by the name of
           its corresponding column in the input dataframe), dictionary of amino acid abbreviations, range of values
           over which to calculate propensity values for continuous property 1, and range of values over which to
           calculate propensity values for continuous property 2
    """
    for prop in [prop_1, prop_2]:
        df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
        df = df.reset_index(drop=True)

    propensities_dict = OrderedDict()

    neighbouring_pairs_list, prop_dicts = gen_neighbouring_pairs_list(
        df, interaction, aa_dict, prop_1, prop_2
    )
    prop_1_dict = prop_dicts[prop_1]
    prop_2_dict = prop_dicts[prop_2]
    propensity_df, frequency_df, normed_frequencies_df = calc_aa_pair_propensities(neighbouring_pairs_list, aa_dict)
    propensity_df = propensity_df.set_index('FASTA', drop=True)

    for aa_1 in list(aa_dict.keys()):
        for aa_2 in list(aa_dict.keys()):
            all_aa_prop_1_list = []
            aa_1_prop_1_list = []
            aa_2_prop_1_list = []
            aa_1_aa_2_prop_1_list = []
            all_aa_prop_2_list = []
            aa_1_prop_2_list = []
            aa_2_prop_2_list = []
            aa_1_aa_2_prop_2_list = []

            for aa_pair in list(prop_1_dict.keys()):
                aas = aa_pair.split('_')[2:]

                all_aa_prop_1_list.append(prop_1_dict[aa_pair][0])
                all_aa_prop_2_list.append(prop_2_dict[aa_pair][0])
                if aa_1 == aas[0]:
                    aa_1_prop_1_list.append(prop_1_dict[aa_pair][0])
                    aa_1_prop_2_list.append(prop_2_dict[aa_pair][0])
                if aa_2 == aas[-1]:
                    aa_2_prop_1_list.append(prop_1_dict[aa_pair][0])  # Want z_coords of aas interacting with aa_2
                    aa_2_prop_2_list.append(prop_2_dict[aa_pair][0])  # Want z_coords of aas interacting with aa_2
                if aa_1 == aas[0] and aa_2 == aas[-1]:
                    aa_1_aa_2_prop_1_list.append(prop_1_dict[aa_pair][0])
                    aa_1_aa_2_prop_2_list.append(prop_2_dict[aa_pair][0])

            if (
                    (not np.isnan(propensity_df[aa_1][aa_2]))
                and (min([len(all_aa_prop_1_list), len(aa_1_prop_1_list),
                          len(aa_2_prop_1_list), len(aa_1_aa_2_prop_1_list),
                          len(all_aa_prop_2_list), len(aa_1_prop_2_list),
                          len(aa_2_prop_2_list), len(aa_1_aa_2_prop_2_list)]) >= 10)
            ):
                print('Propensity for {} to interact with {}'.format(aa_1, aa_2))

                plt.clf()
                all_aa_prop_1 = np.asarray(all_aa_prop_1_list)
                all_aa_prop_1 = all_aa_prop_1.astype(np.float64)
                all_aa_prop_2 = np.asarray(all_aa_prop_2_list)
                all_aa_prop_2 = all_aa_prop_2.astype(np.float64)
                x_all_aa_list, y_all_aa_list, z_all_aa_list = sns.distributions._statsmodels_bivariate_kde(
                    x=all_aa_prop_1, y=all_aa_prop_2, bw='scott', gridsize=100, cut=3,
                    clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
                )

                plt.clf()
                aa_1_prop_1 = np.asarray(aa_1_prop_1_list)
                aa_1_prop_1 = aa_1_prop_1.astype(np.float64)
                aa_1_prop_2 = np.asarray(aa_1_prop_2_list)
                aa_1_prop_2 = aa_1_prop_2.astype(np.float64)
                x_aa_1_list, y_aa_1_list, z_aa_1_list = sns.distributions._statsmodels_bivariate_kde(
                    x=aa_1_prop_1, y=aa_1_prop_2, bw='scott', gridsize=100, cut=3,
                    clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
                )

                plt.clf()
                aa_2_prop_1 = np.asarray(aa_2_prop_1_list)
                aa_2_prop_1 = aa_2_prop_1.astype(np.float64)
                aa_2_prop_2 = np.asarray(aa_2_prop_2_list)
                aa_2_prop_2 = aa_2_prop_2.astype(np.float64)
                x_aa_2_list, y_aa_2_list, z_aa_2_list = sns.distributions._statsmodels_bivariate_kde(
                    x=aa_2_prop_1, y=aa_2_prop_2, bw='scott', gridsize=100, cut=3,
                    clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
                )

                plt.clf()
                aa_1_aa_2_prop_1 = np.asarray(aa_1_aa_2_prop_1_list)
                aa_1_aa_2_prop_1 = aa_1_aa_2_prop_1.astype(np.float64)
                aa_1_aa_2_prop_2 = np.asarray(aa_1_aa_2_prop_2_list)
                aa_1_aa_2_prop_2 = aa_1_aa_2_prop_2.astype(np.float64)
                x_aa_1_aa_2_list, y_aa_1_aa_2_list, z_aa_1_aa_2_list = sns.distributions._statsmodels_bivariate_kde(
                    x=aa_1_aa_2_prop_1, y=aa_1_aa_2_prop_2, bw='scott', gridsize=100, cut=3,
                    clip=[(-np.inf, np.inf), (-np.inf, np.inf)]
                )

                x_coord_lists = [x_all_aa_list, x_aa_1_list, x_aa_2_list]
                y_coord_lists = [y_all_aa_list, y_aa_1_list, y_aa_2_list]
                z_coord_lists = [z_all_aa_list, z_aa_1_list, z_aa_2_list]

                propensities = np.full(z_aa_1_aa_2_list.shape, np.nan)

                for index_1, value_1 in np.ndenumerate(z_aa_1_aa_2_list):
                    column = index_1[0]
                    row = index_1[1]

                    x_aa_1_aa_2 = x_aa_1_aa_2_list[column][row]
                    y_aa_1_aa_2 = y_aa_1_aa_2_list[column][row]
                    z_aa_1_aa_2 = z_aa_1_aa_2_list[column][row]

                    # Finds x,y pairs in overall z distribution for interpolation
                    if (
                            x_aa_1_aa_2 >= cont_prop_1_range[0]
                        and x_aa_1_aa_2 <= cont_prop_1_range[1]
                        and y_aa_1_aa_2 >= cont_prop_2_range[0]
                        and y_aa_1_aa_2 <= cont_prop_2_range[1]
                    ):
                        interpolated_z_vals = []

                        for index_2, value_2 in enumerate(x_coord_lists):
                            x_coord_list = x_coord_lists[index_2][0]
                            y_coord_list = np.transpose(y_coord_lists[index_2])[0]
                            z_coord_list = z_coord_lists[index_2]

                            interpolate_x_indices = []
                            for index_3, value_3 in np.ndenumerate(x_coord_list):
                                index_3 = index_3[0]
                                x_coord = copy.deepcopy(value_3)
                                if x_aa_1_aa_2 == x_coord:
                                    if index_3 < (len(x_coord_list)-1):
                                        interpolate_x_indices = [index_3, index_3+1]
                                    else:
                                        interpolate_x_indices = [index_3-1, index_3]
                                    break
                                elif (
                                          x_coord_list[index_3] < x_aa_1_aa_2
                                      and x_coord_list[index_3+1] > x_aa_1_aa_2
                                ):
                                    interpolate_x_indices = [index_3, index_3+1]
                                    break

                            interpolate_y_indices = []
                            for index_4, value_4 in np.ndenumerate(y_coord_list):
                                index_4 = index_4[0]
                                y_coord = copy.deepcopy(value_4)
                                if y_aa_1_aa_2 == y_coord:
                                    if index_4 < (len(y_coord_list)-1):
                                        interpolate_y_indices = [index_4, index_4+1]
                                    else:
                                        interpolate_y_indices = [index_4-1, index_4]
                                    break
                                elif (
                                          y_coord_list[index_4] < y_aa_1_aa_2
                                      and y_coord_list[index_4+1] > y_aa_1_aa_2
                                ):
                                    interpolate_y_indices = [index_4, index_4+1]
                                    break

                            # Bilinear interpolation
                            x1 = x_coord_list[interpolate_x_indices[0]]
                            x2 = x_coord_list[interpolate_x_indices[1]]
                            y1 = y_coord_list[interpolate_y_indices[0]]
                            y2 = y_coord_list[interpolate_y_indices[1]]
                            z_x1y1 = z_coord_list[interpolate_y_indices[0]][interpolate_x_indices[0]]
                            z_x2y1 = z_coord_list[interpolate_y_indices[0]][interpolate_x_indices[1]]
                            z_x1y2 = z_coord_list[interpolate_y_indices[1]][interpolate_x_indices[0]]
                            z_x2y2 = z_coord_list[interpolate_y_indices[1]][interpolate_x_indices[1]]

                            x1_weight = abs(x2 - x_aa_1_aa_2) / abs(x2 - x1)
                            x2_weight = abs(x_aa_1_aa_2 - x1) / abs(x2 - x1)
                            y1_weight = abs(y2 - y_aa_1_aa_2) / abs(y2 - y1)
                            y2_weight = abs(y_aa_1_aa_2 - y1) / abs(y2 - y1)

                            z_xy1 = (z_x1y1*x1_weight) + (z_x2y1*x2_weight)
                            z_xy2 = (z_x1y2*x1_weight) + (z_x2y2*x2_weight)

                            z = (z_xy1*y1_weight)+(z_xy2*y2_weight)
                            interpolated_z_vals.append(z)

                        z_all_aa = interpolated_z_vals[0]
                        z_aa_1 = interpolated_z_vals[1]
                        z_aa_2 = interpolated_z_vals[2]
                        z_ratio = ((z_aa_1_aa_2 / z_aa_1)
                                   / (z_aa_2 / z_all_aa))
                        propensity = z_ratio * propensity_df[aa_1][aa_2]
                        propensities[column][row] = propensity

                # Plots individual pairwise KDEs used in propensity calculation
                plt.clf()
                fig, ax = plt.subplots(figsize=(15, 5))
                contours_1 = ax.contour(x_all_aa_list, y_all_aa_list, z_all_aa_list, 10, cmap='Blues')
                colourbar_1 = fig.colorbar(contours_1)
                contours_2 = ax.contour(x_aa_1_list, y_aa_1_list, z_aa_1_list, 10, cmap='Oranges')
                colourbar_2 = fig.colorbar(contours_2)
                contours_3 = ax.contour(x_aa_2_list, y_aa_2_list, z_aa_2_list, 10, cmap='Greens')
                colourbar_3 = fig.colorbar(contours_3)
                contours_4 = ax.contour(x_aa_1_aa_2_list, y_aa_1_aa_2_list, z_aa_1_aa_2_list, 10, cmap='RdPu')
                colourbar_4 = fig.colorbar(contours_4)
                plt.show()

                # Plots propensity KDE
                plt.clf()
                fig, ax = plt.subplots(figsize=(8, 5))
                contours_1 = ax.contour(x_aa_1_aa_2_list, y_aa_1_aa_2_list, propensities, 10, cmap='RdPu')
                colourbar_1 = fig.colorbar(contours_1)
                plt.show()
                
                # Updates dictionary of amino acid propensities
                propensities_dict['{}_{}'.format(aa_1, aa_2)] = np.array([x_aa_1_aa_2_list,
                                                                          y_aa_1_aa_2_list,
                                                                          propensities])
                
    return propensities_dict

In [ ]:
def plot_discrete_prop_frequency_kdes(df, cont_prop, discrete_props):
    """
    Plots kernel density estimate of the distribution of a discrete property (e.g. the number of interactions of a
    particular type (e.g. hydrogen bonds) formed by an amino acid)  vs. a continuous property (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via the
           name of the corresponding column in the input dataframe), and list of discrete properties (each discrete
           property is considered individually, in a separate kde plot)
    """
    domain_res_ids_list = []
    for row in range(df.shape[0]):
        domain_res_id = df['domain_ids'][row] + df['res_ids'][row]
        domain_res_ids_list.append(domain_res_id)

    for discrete_prop in discrete_props:
        print(discrete_prop)

        cont_prop_distribution = []
        for row in range(df.shape[0]):
            interacting_domain_neighbours = []

            for res_id in df[discrete_prop][row]:
                domain_res_id = df['domain_ids'][row] + res_id
                # Only considers interactions with residues that are also within the domain
                if domain_res_id in domain_res_ids_list:
                    interacting_domain_neighbours.append(domain_res_id)

            interaction_num = len(interacting_domain_neighbours)
            count = 0
            while count < interaction_num:
                cont_prop_distribution.append(df[cont_prop][row])
                count += 1

        plt.clf()
        plt.figure(figsize=(10, 5))
        sns.distplot(pd.DataFrame({cont_prop: cont_prop_distribution}), hist=False, rug=True,
                     kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.1})
        plt.show()

In [ ]:
def plot_prop_vs_prop(df, cont_prop_1, cont_prop_dict_2, cont_prop_2, bw_1, bw_2):
    """
    Plots the 2D kernel desity estimate of two continuous properties
    Input: dataframe of barrel / sandwich properties, name of first continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), dictionary of amino acid identities vs.
           their values of the second continuous property of interest, the name of the second continuous property
           of interest, the kernel bandwidth of the first property and the kernel bandwidth of the second property
    """
    cont_prop_2_list = ['']*df.shape[0]
    unprocessed_indices = []

    for row in range(df.shape[0]):
        fasta = df['fasta_seq'][row]

        if fasta in cont_prop_dict_2:
            cont_prop_2_list[row] = cont_prop_dict_2[fasta]
        else:
            unprocessed_indices.append(row)

    cont_prop_1_list = [val for index, val in enumerate(df[cont_prop_1].tolist()) if not index in unprocessed_indices]
    cont_prop_2_list = [val for index, val in enumerate(cont_prop_2_list) if not index in unprocessed_indices]

    prop_df = pd.DataFrame({cont_prop_1: cont_prop_1_list,
                            cont_prop_2: cont_prop_2_list})

    plt.clf()
    plt.figure(figsize=(10, 10))
    sns.kdeplot(data=prop_df[cont_prop_1], data2=prop_df[cont_prop_2], bw=(bw_1,bw_2), shade=True, cbar=True)
    # plt.scatter(prop_df[cont_prop_1], prop_df[cont_prop_2], marker='.', c='y')
    plt.show()

In [ ]:
def plot_prop_vs_dihedral(df, cont_prop, dihedral):
    """
    Plots the 2D kernel desity estimate of a continuous property of interest vs. a dihedral angle
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), name of dihedral angle of interest
           (specified via the name of the corresponding column in the input dataframe)
    """
    df = df[~df[dihedral].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    dihedral_df = pd.DataFrame({cont_prop: df[cont_prop].tolist(),
                                dihedral: df[dihedral].tolist()})

    plt.clf()
    plt.figure(figsize=(10, 10))
    sns.kdeplot(data=dihedral_df[cont_prop], data2=dihedral_df[dihedral], bw='scott', shade=True, cbar=True)
    plt.show()

In [ ]:
def calc_dist(df, cont_prop):
    """
    Generates ordered array of a (numeric) property of interest in the input dataframe, without '', 'NaN', 'nan'
    and np.nan values
    Input: dataframe of barrel / sandwich properties, name of property of interest (specified via the name of
           the corresponding column in the input dataframe)
    Returns: ordered numpy array of the property of interest
    """
    cont_prop_vals = df[cont_prop].tolist()
    cont_prop_vals = remove_nan(cont_prop_vals)
    cont_prop_vals = np.array(sorted(cont_prop_vals))

    return cont_prop_vals


def gen_random_array(ordered_array):
    """
    Generates a random array by sampling, with replacement, an input array
    Input: ordered array
    Returns: ordered array (of the same length as the input array) generated by sampling the input array with
             replacement
    """
    random_array = np.empty(len(ordered_array),)

    for num in range(len(ordered_array)):
        rand_index = random.randint(0, (len(ordered_array)-1))
        rand_z_coord = ordered_array[rand_index]
        random_array[num] = rand_z_coord

    return np.sort(random_array)


def run_ks_2samp_test(df, cont_prop, aa, bootstrap):
    """
    Runs the 1D 2 sample Kolmogorov-Smirnov test with scipy to calculate the likelihood that two samples come from
    the same distribution (where sample 1 is the distribution of an individual amino acid with respect to a
    continuous property of interest (typically z-coordinate), and sample 2 is the distribution of all amino acids
    with respect to that property)
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the name of the individual amino acid, and
           whether this test is being run as part of a bootstrap test or not
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    aa_df = df[df['fasta_seq'] == aa]
    aa_df = aa_df.reset_index(drop=True)

    indv_aa_cont_prop_vals = remove_nan(aa_df[cont_prop].tolist())
    indv_aa_cont_prop_vals = np.array(sorted(indv_aa_cont_prop_vals))

    # If performing bootstrap analysis, generate a random array from the df[cont_prop] distribution
    if bootstrap:
        indv_aa_cont_prop_vals = gen_random_array(indv_aa_cont_prop_vals)

    D, p = scipy.stats.ks_2samp(indv_aa_cont_prop_vals, calc_dist(df, cont_prop))

    return D, p


def initial_ks_test(df, cont_prop, ks_func, aa_dict):
    """
    Wrapper to run the 1D 2 sample Kolmogorov-Smirnov test for the original (i.e. not resampled) individual amino
    acid distribution and the overall amino acid distribution
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the function to run the KS test, and
           dictionary of amino acid abbreviations
    Returns: dictionaries of D and p values output from KS test
    """
    D_dict = OrderedDict()
    p_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        D, p = ks_func(df, cont_prop, aa, False)
        D_dict[aa] = [D]
        p_dict[aa] = [p]

    return D_dict, p_dict

def bootstrap_ks_test(df, cont_prop, bootstrap_num, ks_func, aa_dict):
    """
    Wrapper to run the 1D 2 sample Kolmogorov-Smirnov test a specified number of times on different resamplings
    of the original individual amino acid distribution and the overall amino acid distribution
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the number of KS tests to run (i.e. the
           size of the bootstrap), the function to run the KS test, dictionary of amino acid abbreviations
    Returns: dictionaries of D and p values output from KS test
    """
    D_dict = OrderedDict()
    p_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        D_dict[aa] = []
        p_dict[aa] = []

    for num in range(bootstrap_num):
        for aa in list(aa_dict.keys()):
            D, p = ks_func(df, cont_prop, aa, True)
            D_dict[aa].append(D)
            p_dict[aa].append(p)

    return D_dict, p_dict


def draw_plot(plot_type, stat_dict, x_label, y_label):
    """
    Generates swarmplot / violinplot of D / p values vs. amino acid identity
    Input: name of plot ('sns.swarmplot' or 'sns.violinplot'), dictionary of D or p values, label for x-axis 
           (e.g. 'FASTA'), label for y-axis (e.g. 'D' or 'p')
    """
    plt.clf()
    plt.figure(figsize=(15,10))

    x_values = []
    y_values = []

    for key in list(stat_dict.keys()):
        for value in stat_dict[key]:
            x_values.append(key)
            y_values.append(value)

    # "Melted" df of aa vs. stat (either p or D)
    df = pd.DataFrame({'{}'.format(x_label): x_values,
                       '{}'.format(y_label): y_values})

    plot_type(x='{}'.format(x_label), y='{}'.format(y_label), data=df)

    plt.show()


def calc_95_conf_limits(initial_stat_dict, bootstrap_stat_dict):
    """
    Calculates 95% confidence limits on D / p from a bootstrap sample
    Input: dictionary of D / p values for original individual amino acid samples, dictionary of D / p values for
           bootstrapped individual amino acid samples
    Returns: dictionary of 95% confidence intervals for D / p
    """
    conf_int_dict = OrderedDict()

    for aa in list(initial_stat_dict.keys()):
        initial_stat = initial_stat_dict[aa][0]
        bootstrap_stat_values = np.sort(np.array(bootstrap_stat_dict[aa]))

        # Crude method of calculating 95% confidence interval, but good enough for my purposes (to complement
        # "by eye" analysis)
        lower_percentile = np.percentile(bootstrap_stat_values, 2.5)
        upper_percentile = np.percentile(bootstrap_stat_values, 97.5)

        conf_int_dict[aa] = [lower_percentile, initial_stat, upper_percentile]
        
    return conf_int_dict


def iterate_bootstrap_conf_limits(df, cont_prop, ks_func, bootstrap_range, aa_dict):
    """
    Performs KS test with increasing number of bootstrap samples and plots output to allow user to determine when the
    confidence intervals converge (and so the minimum number of bootstrap samples they need to run)
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (typically
           z-coordinate) (specified via the name of the corresponding column in the input dataframe), the function
           to run the KS test, list of numbers of bootstrap samples to test (e.g. [100, 300, 1000, 3000, 10000, 30000,
           100000]), dictionary of amino acid abbreviations
    """
    initial_D_dict, initial_p_dict = initial_ks_test(df, cont_prop, ks_func, aa_dict)

    bootstrap_D_dict = OrderedDict()
    bootstrap_p_dict = OrderedDict()

    for num in bootstrap_range:
        print(num)
        D_dict, p_dict = bootstrap_ks_test(df, cont_prop, num, ks_func, aa_dict)
        bootstrap_D_dict[num] = D_dict
        bootstrap_p_dict[num] = p_dict

    bootstrap_conf_intv_D_dict = OrderedDict()
    bootstrap_conf_intv_p_dict = OrderedDict()
    for num in bootstrap_range:
        conf_int_D_dict = calc_95_conf_limits(initial_D_dict, bootstrap_D_dict[num])
        bootstrap_conf_intv_D_dict[num] = conf_int_D_dict
        conf_int_p_dict = calc_95_conf_limits(initial_p_dict, bootstrap_p_dict[num])
        bootstrap_conf_intv_p_dict[num] = conf_int_p_dict

    for aa in list(aa_dict.keys()):
        aa_bootstrap_D_dict = OrderedDict()
        aa_bootstrap_p_dict = OrderedDict()
        
        for num in bootstrap_range:
            aa_bootstrap_D_dict[num] = bootstrap_conf_intv_D_dict[num][aa]
            aa_bootstrap_p_dict[num] = bootstrap_conf_intv_p_dict[num][aa]
        
        print(aa)
        draw_plot(sns.swarmplot, aa_bootstrap_p_dict, 'Num_bootstrap_samples', 'p')

In [ ]:
def calc_pair_proportions(df, pair_type, aa_dict):
    """
    For every pairwise combination of amino acids listed in the input dictionary of amino acid abbreviations, aa1aa2,
    calculates proportions of amino acid pairs at a location of interest (e.g. HB, or NHB, pairs) that are of that
    identity. Note no pair inversion.
    Input: dataframe of sandwich / barrel properties, name of amino acid pair type of interest (specified via the
           name of its corresponding column in the input dataframe), and dictionary of amino acid abbreviations
    Returns: dictionary of aa1aa2 pair count, total number of amino acid pairs of type of interest
    """
    neighbouring_pairs_list = gen_neighbouring_pairs_list(df, pair_type, aa_dict)

    pairs_count_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        pairs_count_dict[aa_pair] = 0

    for aa_pair in neighbouring_pairs_list:
        aa_1 = aa_pair.split('_')[0]
        aa_2 = aa_pair.split('_')[1]
        aa_pair_fwd = copy.deepcopy(aa_pair)
        aa_pair_rev = aa_pair_fwd[::-1]

        if (    aa_1 != aa_2
            and aa_pair_fwd in list(pairs_count_dict.keys())
            and aa_pair_rev in list(pairs_count_dict.keys())
        ):
            sys.exit('Error with standard error of proporton calculation')

        if aa_pair_fwd in list(pairs_count_dict.keys()):
            pairs_count_dict[aa_pair_fwd] += 1
        elif aa_pair_rev in list(pairs_count_dict.keys()):
            pairs_count_dict[aa_pair_rev] += 1

    pairs_total = len(neighbouring_pairs_list)
    for aa_pair in list(pairs_count_dict.keys()):
        pairs_count_dict[aa_pair] = pairs_count_dict[aa_pair] / pairs_total

    return pairs_count_dict, pairs_total


def gen_dek_sandwich_dicts(aa_dict, hb_or_nhb):
    """
    Reads beta-sandwich HB / NHB count data from Dek's 1998 paper (which is stored in two separate csv files in
    the BetaStats directory) into a dictionary
    Input: dictionary of amino acid abbreviations, and whether to read in HB (hb_or_nhb = 'hb') or NHB
           (hb_or_nhb = 'nhb') count data
    Returns: dictionary of amino acid pair proportions, total number of HB (hb_or_nhb = 'hb') or NHB
             (hb_or_nhb = 'nhb') amino acid pairs
    """
    pairs_count_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        pairs_count_dict[aa_pair] = 0
        
    file_name = 'Beta_{}_count_data_Hutchinson_et_al_1998.csv'.format(hb_or_nhb.upper())
    df = pd.read_csv(file_name, header=0, index_col=0)
    for col in df.columns.values.tolist():
        df[col] = pd.to_numeric(df[col].tolist())

    pairs_total = 0
    for aa_pair in list(pairs_count_dict.keys()):
        aa_1 = aa_pair.split('_')[0]
        aa_2 = aa_pair.split('_')[1]

        count = df[aa_1][aa_2]
        pairs_total += count
        pairs_count_dict[aa_pair] = count

    for aa_pair in list(pairs_count_dict.keys()):
        pairs_count_dict[aa_pair] = pairs_count_dict[aa_pair] / pairs_total

    return pairs_count_dict, pairs_total


def calc_std_error_proportion(aa_dict, hb_pairs_count_dict, nhb_pairs_count_dict, hb_pairs_total,
                              nhb_pairs_total):
    """
    Calculates standard error of proportion of amino acid pairs that form HB as compared with NHB pairs. These
    results can be used to determine if there is a significant difference between the proportion of aa1aa2 pairs
    at HB positions as compared to NHB positions.
    Input: dictionary of amino acid abbreviations, dictionary of hydrogen bonding pair probabilities, dictionary
           of non-hydrogen-bonding pair probabilities, total number of hydrogen bonding pairs, total number of
           non-hydrogen-bonding pairs
    dataframe of barrel / sandwich properties, dictionary of amino acid abbreviations
    Returns: dictionary of z scores, dictionary of ratios of proportion of aa1aa2 pairs at HB sites to proportion
             of aa1aa2 pairs at NHB sites
    """
    z_score_dict = OrderedDict()
    ratios_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        z_score_dict[aa_pair] = np.nan
        ratios_dict[aa_pair] = np.nan

    for aa_pair in list(hb_pairs_count_dict):
        p_hb = hb_pairs_count_dict[aa_pair]
        p_nhb = nhb_pairs_count_dict[aa_pair]
        
        if p_hb > 0 and p_nhb > 0:
            variance_hb = (p_hb*(1-p_hb)) / hb_pairs_total
            variance_nhb = (p_nhb*(1-p_nhb)) / nhb_pairs_total

            std_dev = (variance_hb + variance_nhb)**0.5
            z = (p_hb - p_nhb) / std_dev
            z_score_dict[aa_pair] = z
            
            ratios_dict[aa_pair] = p_hb / p_nhb

    return z_score_dict, ratios_dict


def print_significant_z_scores(z_score_dict):
    """
    Prints z-scores above / below 95 and 99% confidence limits
    Input: dictionary of z scores
    """
    for aa_pair, score in z_score_dict.items():
        if score > 2.58:
            print('Outside 99% confidence limit, favours HB: {}'.format(aa_pair))
        elif 1.96 < score <= 2.58:
            print('Outside 95% confidence limit, favours HB: {}'.format(aa_pair))
        elif score < -2.58:
            print('Outside 99% confidence limit, favours NHB: {}'.format(aa_pair))
        elif -2.58 <= score < -1.96:
            print('Outside 95% confidence limit, favours NHB: {}'.format(aa_pair))


def calc_z_diff(z_score_dict, dek_sandwich_z_dict):
    """
    Calculates the difference in area underneath the normal distribution curve between two z-scores
    Input: first dictionary of z scores, second dictionary of z scores
    Returns: dictionary of differences
    """
    z_diff_dict = OrderedDict()

    for aa_pair in list(z_score_dict.keys()):
        z_score_old = dek_sandwich_z_dict[aa_pair]
        z_score_new = z_score_dict[aa_pair]
        
        p_old = scipy.stats.norm.cdf(z_score_old)
        p_new = scipy.stats.norm.cdf(z_score_new)
        
        area_diff = abs(p_new - p_old)
        z_diff_dict[aa_pair] = area_diff

    return z_diff_dict

## Load dataframe

In [ ]:
barrel_file_loc = (
    '/Users/ks17361/Lab_work_DW/Beta_structure/Parametrisation/My_beta_sandwich_and_barrel_datasets/'
    'Beta_Datasets_30_10_2018/seqid_40/CATH_2.40.160_2.40.170_2.40.230_2.40.128_resn_2.5_rfac_0.3_ba/'
    'Beta_res_dataframe.pkl'
)

In [ ]:
barrel_df = pd.read_pickle(barrel_file_loc)
barrel_df = barrel_df[barrel_df['fasta_seq'].isin(list(aa_dict.keys()))]
barrel_df = barrel_df.reset_index(drop=True)

In [ ]:
barrel_df.head()

### Amino acid distribution in beta-barrels

In [ ]:
barrel_distribution = calc_distribution(barrel_df, aa_dict)

plt.clf()
plt.figure(figsize=(20, 10))
sns.barplot(x='FASTA', y='Normalised frequency', data=barrel_distribution)
plt.show()

In [ ]:
for aa in list(aa_dict.keys()):
    print(aa + ': ' + str(barrel_df['fasta_seq'].tolist().count(aa)))

Too few Cys residues for meaningful analysis => discounted from further analysis

In [ ]:
aa_dict_no_cys = copy.deepcopy(aa_dict)
aa_dict_no_cys.pop('C')
aa_dict_no_cys

In [ ]:
barrel_df = barrel_df[barrel_df['fasta_seq'] != 'C']
barrel_df = barrel_df.reset_index(drop=True)

barrel_distribution = calc_distribution(barrel_df, aa_dict)

plt.clf()
plt.figure(figsize=(20, 10))
sns.barplot(x='FASTA', y='Normalised frequency', data=barrel_distribution)
plt.show()

In [ ]:
for aa in list(aa_dict_no_cys.keys()):
    print(aa + ': ' + str(barrel_df['fasta_seq'].tolist().count(aa)))

How does this distribution compare to those previously determined?

In [ ]:
levitt_dict = OrderedDict({'A': 0.900, 'R': 0.990, 'N': 0.760, 'D': 0.720, 'C': 0.740, 'Q': 0.800, 'E': 0.750,
                           'G': 0.920, 'H': 1.080, 'I': 1.450, 'L': 1.020, 'K': 0.770, 'M': 0.970, 'F': 1.320,
                           'P': 0.640, 'S': 0.950, 'T': 1.210, 'W': 1.140, 'Y': 1.250, 'V': 1.490})
levitt_df = pd.DataFrame({'FASTA': list(levitt_dict.keys()),
                          'Propensity': list(levitt_dict.values())})

plt.clf()
plt.figure(figsize=(20, 10))
sns.barplot(x='FASTA', y='Propensity', data=levitt_df)
plt.show()

In [ ]:
chou_fasman_dict = OrderedDict({'A': 0.830, 'R': 0.930, 'N': 0.890, 'D': 0.540, 'C': 1.190, 'Q': 1.100, 'E': 0.370,
                                'G': 0.750, 'H': 0.870, 'I': 1.600, 'L': 1.300, 'K': 0.740, 'M': 1.050, 'F': 1.380, 
                                'P': 0.550, 'S': 0.750, 'T': 1.190, 'W': 1.370, 'Y': 1.470, 'V': 1.700})
chou_fasman_df = pd.DataFrame({'FASTA': list(chou_fasman_dict.keys()),
                               'Propensity': list(chou_fasman_dict.values())})

plt.clf()
plt.figure(figsize=(20, 10))
sns.barplot(x='FASTA', y='Propensity', data=chou_fasman_df)
plt.show()

## Discrete propensity calculations

### Interior / exterior propensity

In [ ]:
int_ext_propensity, int_ext_frequency, int_ext_normed_frequencies = calc_indv_property_propensities(
    barrel_df, 'int_ext', aa_dict_no_cys
)

In [ ]:
int_ext_propensity

In [ ]:
int_ext_frequency

In [ ]:
plot_bar_graphs(int_ext_propensity, 'Propensity')

In [ ]:
plot_bar_graphs(int_ext_frequency, 'Frequency')

In [ ]:
plot_heat_map(int_ext_propensity)

In [ ]:
plot_heat_map(int_ext_frequency)

In [ ]:
(
    int_ext_bootstrap_propensity_dict, int_ext_bootstrap_frequency_dict, int_ext_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'int_ext', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_indv_property_propensities,
    int_ext_propensity, int_ext_frequency, int_ext_normed_frequencies
)

In [ ]:
(
    int_ext_conf_intv_propensity_dict, int_ext_conf_intv_frequency_dict, int_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'int_ext', aa_dict_no_cys, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_int_ext_propensity_dict, upper_percentile_int_ext_propensity_dict = gen_95_conf_intervals(
    int_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    int_ext_propensity, lower_percentile_int_ext_propensity_dict, upper_percentile_int_ext_propensity_dict,
    'Propensity'
)

In [ ]:
lower_percentile_int_ext_frequency_dict, upper_percentile_int_ext_frequency_dict = gen_95_conf_intervals(
    int_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    int_ext_frequency, lower_percentile_int_ext_frequency_dict, upper_percentile_int_ext_frequency_dict, 'Frequency'
)

### Transmembrane / external propensity

In [ ]:
tm_ext_propensity, tm_ext_frequency, tm_ext_normed_frequencies = calc_indv_property_propensities(
    barrel_df, 'tm_ext', aa_dict_no_cys
)

In [ ]:
tm_ext_propensity

In [ ]:
tm_ext_frequency

In [ ]:
plot_bar_graphs(tm_ext_propensity, 'Propensity')

In [ ]:
plot_bar_graphs(tm_ext_frequency, 'Frequency')

In [ ]:
plot_heat_map(tm_ext_propensity)

In [ ]:
plot_heat_map(tm_ext_frequency)

In [ ]:
(
    tm_ext_bootstrap_propensity_dict, tm_ext_bootstrap_frequency_dict, tm_ext_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'tm_ext', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_indv_property_propensities, tm_ext_propensity,
    tm_ext_frequency, tm_ext_normed_frequencies
)

In [ ]:
(
    tm_ext_conf_intv_propensity_dict, tm_ext_conf_intv_frequency_dict, tm_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'tm_ext', aa_dict_no_cys, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_tm_ext_propensity_dict, upper_percentile_tm_ext_propensity_dict = gen_95_conf_intervals(
    tm_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    tm_ext_propensity, lower_percentile_tm_ext_propensity_dict, upper_percentile_tm_ext_propensity_dict,
    'Propensity'
)

In [ ]:
lower_percentile_tm_ext_frequency_dict, upper_percentile_tm_ext_frequency_dict = gen_95_conf_intervals(
    tm_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    tm_ext_frequency, lower_percentile_tm_ext_frequency_dict, upper_percentile_tm_ext_frequency_dict, 'Frequency'
)

### All combinations propensities

In [ ]:
(int_ext_tm_ext_propensity, int_ext_tm_ext_frequency, int_ext_tm_ext_normed_frequencies
) = calc_combined_property_propensities(
    barrel_df, ['int_ext', 'tm_ext'], aa_dict_no_cys
)

In [ ]:
int_ext_tm_ext_propensity

In [ ]:
int_ext_tm_ext_frequency

In [ ]:
plot_bar_graphs(int_ext_tm_ext_propensity, 'Propensity')

In [ ]:
plot_bar_graphs(int_ext_tm_ext_frequency, 'Frequency')

In [ ]:
plot_heat_map(int_ext_tm_ext_propensity)

In [ ]:
plot_heat_map(int_ext_tm_ext_frequency)

In [ ]:
(
    int_ext_tm_ext_bootstrap_propensity_dict, int_ext_tm_ext_bootstrap_frequency_dict,
    int_ext_tm_ext_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, ['int_ext', 'tm_ext'], aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_combined_property_propensities,
    int_ext_tm_ext_propensity, int_ext_tm_ext_frequency, int_ext_tm_ext_normed_frequencies
)

In [ ]:
(
    int_ext_tm_ext_conf_intv_propensity_dict, int_ext_tm_ext_conf_intv_frequency_dict,
    int_ext_tm_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, ['int_ext', 'tm_ext'], aa_dict_no_cys, 300, calc_combined_property_propensities
)

In [ ]:
(
    lower_percentile_int_ext_tm_ext_propensity_dict, upper_percentile_int_ext_tm_ext_propensity_dict
) = gen_95_conf_intervals(
    int_ext_tm_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    int_ext_tm_ext_propensity, lower_percentile_int_ext_tm_ext_propensity_dict,
    upper_percentile_int_ext_tm_ext_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_int_ext_tm_ext_frequency_dict, upper_percentile_int_ext_tm_ext_frequency_dict
) = gen_95_conf_intervals(
    int_ext_tm_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    int_ext_tm_ext_frequency, lower_percentile_int_ext_tm_ext_frequency_dict,
    upper_percentile_int_ext_tm_ext_frequency_dict, 'Frequency'
)

### Phi psi discrete bin propensities

In [ ]:
discrete_phi_psi_cluster_coords, discrete_phi_psi_cluster_dataframe = calc_voronoi_points(barrel_df, 'phi', 'psi', 25)

In [ ]:
vor = scipy.spatial.Voronoi(discrete_phi_psi_cluster_coords)

plt.clf()
scipy.spatial.voronoi_plot_2d(vor)
plt.show()

In [ ]:
(discrete_phi_psi_propensity, discrete_phi_psi_frequency, discrete_phi_psi_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(barrel_df, 'phi', 'psi', aa_dict_no_cys, discrete_phi_psi_cluster_coords)

In [ ]:
discrete_phi_psi_propensity

In [ ]:
discrete_phi_psi_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_propensity)

In [ ]:
plot_heat_map(discrete_phi_psi_frequency)

In [ ]:
(
    discrete_phi_psi_bootstrap_propensity_dict, discrete_phi_psi_bootstrap_frequency_dict,
    discrete_phi_psi_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, ['phi', 'psi'], aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_propensity, discrete_phi_psi_frequency, discrete_phi_psi_normed_frequencies, 25
)

In [ ]:
(
    discrete_phi_psi_conf_intv_propensity_dict, discrete_phi_psi_conf_intv_frequency_dict,
    discrete_phi_psi_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, ['phi', 'psi'], aa_dict_no_cys, 300, calc_discrete_2d_indv_aa_propensities
)

In [ ]:
lower_percentile_discrete_phi_psi_propensity_dict, upper_percentile_discrete_phi_psi_propensity_dict = gen_95_conf_intervals(
    discrete_phi_psi_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_propensity, lower_percentile_discrete_phi_psi_propensity_dict,
    upper_percentile_discrete_phi_psi_propensity_dict, 'Propensity'
)

In [ ]:
lower_percentile_discrete_phi_psi_frequency_dict, upper_percentile_discrete_phi_psi_frequency_dict = gen_95_conf_intervals(
    discrete_phi_psi_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_frequency, lower_percentile_discrete_phi_psi_frequency_dict,
    upper_percentile_discrete_phi_psi_frequency_dict, 'Frequency'
)

Interior surface

In [ ]:
discrete_phi_psi_int_cluster_coords, discrete_phi_psi_int_cluster_dataframe = calc_voronoi_points(
    int_barrel_df, 'phi', 'psi', 25
)

In [ ]:
int_vor = scipy.spatial.Voronoi(discrete_phi_psi_int_cluster_coords)

plt.clf()
scipy.spatial.voronoi_plot_2d(int_vor)
plt.show()

In [ ]:
(discrete_phi_psi_int_propensity, discrete_phi_psi_int_frequency, discrete_phi_psi_int_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(
    int_barrel_df, 'phi', 'psi', aa_dict_no_cys, discrete_phi_psi_int_cluster_coords
)

In [ ]:
discrete_phi_psi_int_propensity

In [ ]:
discrete_phi_psi_int_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_int_propensity)

In [ ]:
plot_heat_map(discrete_phi_psi_int_frequency)

In [ ]:
(
    discrete_phi_psi_int_bootstrap_propensity_dict, discrete_phi_psi_int_bootstrap_frequency_dict,
    discrete_phi_psi_int_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, ['phi', 'psi'], aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_int_propensity, discrete_phi_psi_int_frequency, discrete_phi_psi_int_normed_frequencies, 25
)

In [ ]:
(
    discrete_phi_psi_int_conf_intv_propensity_dict, discrete_phi_psi_int_conf_intv_frequency_dict,
    discrete_phi_psi_int_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, ['phi', 'psi'], aa_dict_no_cys, 300, calc_discrete_2d_indv_aa_propensities
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_int_propensity_dict, upper_percentile_discrete_phi_psi_int_propensity_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_int_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_int_propensity, lower_percentile_discrete_phi_psi_int_propensity_dict,
    upper_percentile_discrete_phi_psi_int_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_int_frequency_dict, upper_percentile_discrete_phi_psi_int_frequency_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_int_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_int_frequency, lower_percentile_discrete_phi_psi_int_frequency_dict,
    upper_percentile_discrete_phi_psi_int_frequency_dict, 'Frequency'
)

Exterior surface

In [ ]:
discrete_phi_psi_ext_cluster_coords, discrete_phi_psi_ext_cluster_dataframe = calc_voronoi_points(
    ext_barrel_df, 'phi', 'psi', 25
)

In [ ]:
ext_vor = scipy.spatial.Voronoi(discrete_phi_psi_ext_cluster_coords)

plt.clf()
scipy.spatial.voronoi_plot_2d(ext_vor)
plt.show()

In [ ]:
(discrete_phi_psi_ext_propensity, discrete_phi_psi_ext_frequency, discrete_phi_psi_ext_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(
    ext_barrel_df, 'phi', 'psi', aa_dict_no_cys, discrete_phi_psi_ext_cluster_coords
)

In [ ]:
discrete_phi_psi_ext_propensity

In [ ]:
discrete_phi_psi_ext_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_ext_propensity)

In [ ]:
plot_heat_map(discrete_phi_psi_ext_frequency)

In [ ]:
(
    discrete_phi_psi_ext_bootstrap_propensity_dict, discrete_phi_psi_ext_bootstrap_frequency_dict,
    discrete_phi_psi_ext_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, ['phi', 'psi'], aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_ext_propensity, discrete_phi_psi_ext_frequency, discrete_phi_psi_ext_normed_frequencies, 25
)

In [ ]:
(
    discrete_phi_psi_ext_conf_intv_propensity_dict, discrete_phi_psi_ext_conf_intv_frequency_dict,
    discrete_phi_psi_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, ['phi', 'psi'], aa_dict_no_cys, 300, calc_discrete_2d_indv_aa_propensities
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_ext_propensity_dict, upper_percentile_discrete_phi_psi_ext_propensity_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_ext_propensity, lower_percentile_discrete_phi_psi_ext_propensity_dict,
    upper_percentile_discrete_phi_psi_ext_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_ext_frequency_dict, upper_percentile_discrete_phi_psi_ext_frequency_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_ext_frequency, lower_percentile_discrete_phi_psi_ext_frequency_dict,
    upper_percentile_discrete_phi_psi_ext_frequency_dict, 'Frequency'
)

### Amino acid pairs propensities

In [ ]:
int_barrel_df = barrel_df[barrel_df['int_ext'] == 'interior']
int_barrel_df = int_barrel_df.reset_index(drop=True)

ext_barrel_df = barrel_df[barrel_df['int_ext'] == 'exterior']
ext_barrel_df = ext_barrel_df.reset_index(drop=True)

#### Hydrogen bonding pairs

Both barrel surfaces

In [ ]:
hb_aa_pairs = gen_neighbouring_pairs_list(barrel_df, 'hb_pairs', aa_dict_no_cys)
(hb_aa_pair_propensity, hb_aa_pair_frequency, hb_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_aa_pairs, aa_dict_no_cys)

In [ ]:
hb_aa_pair_propensity

In [ ]:
hb_aa_pair_frequency

In [ ]:
plot_heat_map(hb_aa_pair_propensity)

In [ ]:
plot_heat_map(hb_aa_pair_frequency)

In [ ]:
(
    hb_aa_pair_bootstrap_propensity_dict, hb_aa_pair_bootstrap_frequency_dict,
    hb_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'hb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_aa_pair_propensity, hb_aa_pair_frequency, hb_aa_pair_normed_frequencies
)

In [ ]:
(
    hb_aa_pair_conf_intv_propensity_dict, hb_aa_pair_conf_intv_frequency_dict,
    hb_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'hb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_aa_pair_propensity_dict, upper_percentile_hb_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    hb_aa_pair_propensity, lower_percentile_hb_aa_pair_propensity_dict,
    upper_percentile_hb_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_hb_aa_pair_frequency_dict, upper_percentile_hb_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    hb_aa_pair_frequency, lower_percentile_hb_aa_pair_frequency_dict,
    upper_percentile_hb_aa_pair_frequency_dict, 'Frequency'
)

Interior barrel surface

In [ ]:
hb_int_aa_pairs = gen_neighbouring_pairs_list(int_barrel_df, 'hb_pairs', aa_dict_no_cys)
(hb_int_aa_pair_propensity, hb_int_aa_pair_frequency, hb_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_int_aa_pairs, aa_dict_no_cys)

In [ ]:
hb_int_aa_pair_propensity

In [ ]:
hb_int_aa_pair_frequency

In [ ]:
plot_heat_map(hb_int_aa_pair_propensity)

In [ ]:
plot_heat_map(hb_int_aa_pair_frequency)

In [ ]:
(
    hb_int_aa_pair_bootstrap_propensity_dict, hb_int_aa_pair_bootstrap_frequency_dict,
    hb_int_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, 'hb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_int_aa_pair_propensity, hb_int_aa_pair_frequency, hb_int_aa_pair_normed_frequencies
)

In [ ]:
(
    hb_int_aa_pair_conf_intv_propensity_dict, hb_int_aa_pair_conf_intv_frequency_dict,
    hb_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, 'hb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_int_aa_pair_propensity_dict, upper_percentile_hb_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    hb_int_aa_pair_propensity, lower_percentile_hb_int_aa_pair_propensity_dict,
    upper_percentile_hb_int_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_hb_int_aa_pair_frequency_dict, upper_percentile_hb_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    hb_int_aa_pair_frequency, lower_percentile_hb_int_aa_pair_frequency_dict,
    upper_percentile_hb_int_aa_pair_frequency_dict, 'Frequency'
)

Exterior barrel surface

In [ ]:
hb_ext_aa_pairs = gen_neighbouring_pairs_list(ext_barrel_df, 'hb_pairs', aa_dict_no_cys)
(hb_ext_aa_pair_propensity, hb_ext_aa_pair_frequency, hb_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_ext_aa_pairs, aa_dict_no_cys)

In [ ]:
hb_ext_aa_pair_propensity

In [ ]:
hb_ext_aa_pair_frequency

In [ ]:
plot_heat_map(hb_ext_aa_pair_propensity)

In [ ]:
plot_heat_map(hb_ext_aa_pair_frequency)

In [ ]:
(
    hb_ext_aa_pair_bootstrap_propensity_dict, hb_ext_aa_pair_bootstrap_frequency_dict,
    hb_ext_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, 'hb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_ext_aa_pair_propensity, hb_ext_aa_pair_frequency, hb_ext_aa_pair_normed_frequencies
)

In [ ]:
(
    hb_ext_aa_pair_conf_intv_propensity_dict, hb_ext_aa_pair_conf_intv_frequency_dict,
    hb_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, 'hb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_ext_aa_pair_propensity_dict, upper_percentile_hb_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    hb_ext_aa_pair_propensity, lower_percentile_hb_ext_aa_pair_propensity_dict,
    upper_percentile_hb_ext_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_hb_ext_aa_pair_frequency_dict, upper_percentile_hb_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    hb_ext_aa_pair_frequency, lower_percentile_hb_ext_aa_pair_frequency_dict,
    upper_percentile_hb_ext_aa_pair_frequency_dict, 'Frequency'
)

#### Non- hydrogen bonding pairs

Both barrel surfaces

In [ ]:
nhb_aa_pairs = gen_neighbouring_pairs_list(barrel_df, 'nhb_pairs', aa_dict_no_cys)
(nhb_aa_pair_propensity, nhb_aa_pair_frequency, nhb_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_aa_pairs, aa_dict_no_cys)

In [ ]:
nhb_aa_pair_propensity

In [ ]:
nhb_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_aa_pair_propensity)

In [ ]:
plot_heat_map(nhb_aa_pair_frequency)

In [ ]:
(
    nhb_aa_pair_bootstrap_propensity_dict, nhb_aa_pair_bootstrap_frequency_dict,
    nhb_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'nhb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_aa_pair_propensity, nhb_aa_pair_frequency, nhb_aa_pair_normed_frequencies
)

In [ ]:
(
    nhb_aa_pair_conf_intv_propensity_dict, nhb_aa_pair_conf_intv_frequency_dict,
    nhb_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'nhb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_aa_pair_propensity_dict, upper_percentile_nhb_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    nhb_aa_pair_propensity, lower_percentile_nhb_aa_pair_propensity_dict,
    upper_percentile_nhb_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_nhb_aa_pair_frequency_dict, upper_percentile_nhb_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    nhb_aa_pair_frequency, lower_percentile_nhb_aa_pair_frequency_dict,
    upper_percentile_nhb_aa_pair_frequency_dict, 'Frequency'
)

Interior barrel surface

In [ ]:
nhb_int_aa_pairs = gen_neighbouring_pairs_list(int_barrel_df, 'nhb_pairs', aa_dict_no_cys)
(nhb_int_aa_pair_propensity, nhb_int_aa_pair_frequency, nhb_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_int_aa_pairs, aa_dict_no_cys)

In [ ]:
nhb_int_aa_pair_propensity

In [ ]:
nhb_int_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_int_aa_pair_propensity)

In [ ]:
plot_heat_map(nhb_int_aa_pair_frequency)

In [ ]:
(
    nhb_int_aa_pair_bootstrap_propensity_dict, nhb_int_aa_pair_bootstrap_frequency_dict,
    nhb_int_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, 'nhb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_int_aa_pair_propensity, nhb_int_aa_pair_frequency, nhb_int_aa_pair_normed_frequencies
)

In [ ]:
(
    nhb_int_aa_pair_conf_intv_propensity_dict, nhb_int_aa_pair_conf_intv_frequency_dict,
    nhb_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, 'nhb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_int_aa_pair_propensity_dict, upper_percentile_nhb_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    nhb_int_aa_pair_propensity, lower_percentile_nhb_int_aa_pair_propensity_dict,
    upper_percentile_nhb_int_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_nhb_int_aa_pair_frequency_dict, upper_percentile_nhb_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    nhb_int_aa_pair_frequency, lower_percentile_nhb_int_aa_pair_frequency_dict,
    upper_percentile_nhb_int_aa_pair_frequency_dict, 'Frequency'
)

Exterior barrel surface

In [ ]:
nhb_ext_aa_pairs = gen_neighbouring_pairs_list(ext_barrel_df, 'nhb_pairs', aa_dict_no_cys)
(nhb_ext_aa_pair_propensity, nhb_ext_aa_pair_frequency, nhb_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_ext_aa_pairs, aa_dict_no_cys)

In [ ]:
nhb_ext_aa_pair_propensity

In [ ]:
nhb_ext_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_ext_aa_pair_propensity)

In [ ]:
plot_heat_map(nhb_ext_aa_pair_frequency)

In [ ]:
(
    nhb_ext_aa_pair_bootstrap_propensity_dict, nhb_ext_aa_pair_bootstrap_frequency_dict,
    nhb_ext_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, 'nhb_pairs', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_ext_aa_pair_propensity, nhb_ext_aa_pair_frequency, nhb_ext_aa_pair_normed_frequencies
)

In [ ]:
(
    nhb_ext_aa_pair_conf_intv_propensity_dict, nhb_ext_aa_pair_conf_intv_frequency_dict,
    nhb_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, 'nhb_pairs', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_ext_aa_pair_propensity_dict, upper_percentile_nhb_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    nhb_ext_aa_pair_propensity, lower_percentile_nhb_ext_aa_pair_propensity_dict,
    upper_percentile_nhb_ext_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_nhb_ext_aa_pair_frequency_dict, upper_percentile_nhb_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    nhb_ext_aa_pair_frequency, lower_percentile_nhb_ext_aa_pair_frequency_dict,
    upper_percentile_nhb_ext_aa_pair_frequency_dict, 'Frequency'
)

#### Minus 2 pairs

Both barrel surfaces

In [ ]:
minus2_aa_pairs = gen_neighbouring_pairs_list(barrel_df, 'minus_2', aa_dict_no_cys)
(minus2_aa_pair_propensity, minus2_aa_pair_frequency, minus2_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_aa_pairs, aa_dict_no_cys)

In [ ]:
minus2_aa_pair_propensity

In [ ]:
minus2_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_aa_pair_propensity)

In [ ]:
plot_heat_map(minus2_aa_pair_frequency)

In [ ]:
(
    minus2_aa_pair_bootstrap_propensity_dict, minus2_aa_pair_bootstrap_frequency_dict,
    minus2_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'minus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_aa_pair_propensity, minus2_aa_pair_frequency, minus2_aa_pair_normed_frequencies
)

In [ ]:
(
    minus2_aa_pair_conf_intv_propensity_dict, minus2_aa_pair_conf_intv_frequency_dict,
    minus2_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'minus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_aa_pair_propensity_dict, upper_percentile_minus2_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    minus2_aa_pair_propensity, lower_percentile_minus2_aa_pair_propensity_dict,
    upper_percentile_minus2_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_minus2_aa_pair_frequency_dict, upper_percentile_minus2_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    minus2_aa_pair_frequency, lower_percentile_minus2_aa_pair_frequency_dict,
    upper_percentile_minus2_aa_pair_frequency_dict, 'Frequency'
)

Interior barrel surface

In [ ]:
minus2_int_aa_pairs = gen_neighbouring_pairs_list(int_barrel_df, 'minus_2', aa_dict_no_cys)
(minus2_int_aa_pair_propensity, minus2_int_aa_pair_frequency, minus2_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_int_aa_pairs, aa_dict_no_cys)

In [ ]:
minus2_int_aa_pair_propensity

In [ ]:
minus2_int_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_int_aa_pair_propensity)

In [ ]:
plot_heat_map(minus2_int_aa_pair_frequency)

In [ ]:
(
    minus2_int_aa_pair_bootstrap_propensity_dict, minus2_int_aa_pair_bootstrap_frequency_dict,
    minus2_int_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, 'minus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_int_aa_pair_propensity, minus2_int_aa_pair_frequency, minus2_int_aa_pair_normed_frequencies
)

In [ ]:
(
    minus2_int_aa_pair_conf_intv_propensity_dict, minus2_int_aa_pair_conf_intv_frequency_dict,
    minus2_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, 'minus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_int_aa_pair_propensity_dict, upper_percentile_minus2_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus2_int_aa_pair_propensity, lower_percentile_minus2_int_aa_pair_propensity_dict,
    upper_percentile_minus2_int_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_minus2_int_aa_pair_frequency_dict, upper_percentile_minus2_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus2_int_aa_pair_frequency, lower_percentile_minus2_int_aa_pair_frequency_dict,
    upper_percentile_minus2_int_aa_pair_frequency_dict, 'Frequency'
)

Exterior barrel surface

In [ ]:
minus2_ext_aa_pairs = gen_neighbouring_pairs_list(ext_barrel_df, 'minus_2', aa_dict_no_cys)
(minus2_ext_aa_pair_propensity, minus2_ext_aa_pair_frequency, minus2_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_ext_aa_pairs, aa_dict_no_cys)

In [ ]:
minus2_ext_aa_pair_propensity

In [ ]:
minus2_ext_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_ext_aa_pair_propensity)

In [ ]:
plot_heat_map(minus2_ext_aa_pair_frequency)

In [ ]:
(
    minus2_ext_aa_pair_bootstrap_propensity_dict, minus2_ext_aa_pair_bootstrap_frequency_dict,
    minus2_ext_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, 'minus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_ext_aa_pair_propensity, minus2_ext_aa_pair_frequency, minus2_ext_aa_pair_normed_frequencies
)

In [ ]:
(
    minus2_ext_aa_pair_conf_intv_propensity_dict, minus2_ext_aa_pair_conf_intv_frequency_dict,
    minus2_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, 'minus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_ext_aa_pair_propensity_dict, upper_percentile_minus2_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus2_ext_aa_pair_propensity, lower_percentile_minus2_ext_aa_pair_propensity_dict,
    upper_percentile_minus2_ext_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_minus2_ext_aa_pair_frequency_dict, upper_percentile_minus2_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus2_ext_aa_pair_frequency, lower_percentile_minus2_ext_aa_pair_frequency_dict,
    upper_percentile_minus2_ext_aa_pair_frequency_dict, 'Frequency'
)

#### Plus 2 pairs

Both barrel surfaces

In [ ]:
plus2_aa_pairs = gen_neighbouring_pairs_list(barrel_df, 'plus_2', aa_dict_no_cys)
(plus2_aa_pair_propensity, plus2_aa_pair_frequency, plus2_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_aa_pairs, aa_dict_no_cys)

In [ ]:
plus2_aa_pair_propensity

In [ ]:
plus2_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_aa_pair_propensity)

In [ ]:
plot_heat_map(plus2_aa_pair_frequency)

In [ ]:
(
    plus2_aa_pair_bootstrap_propensity_dict, plus2_aa_pair_bootstrap_frequency_dict,
    plus2_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'plus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_aa_pair_propensity, plus2_aa_pair_frequency, plus2_aa_pair_normed_frequencies
)

In [ ]:
(
    plus2_aa_pair_conf_intv_propensity_dict, plus2_aa_pair_conf_intv_frequency_dict,
    plus2_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'plus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_aa_pair_propensity_dict, upper_percentile_plus2_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_aa_pair_propensity, lower_percentile_plus2_aa_pair_propensity_dict,
    upper_percentile_plus2_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_plus2_aa_pair_frequency_dict, upper_percentile_plus2_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_aa_pair_frequency, lower_percentile_plus2_aa_pair_frequency_dict,
    upper_percentile_plus2_aa_pair_frequency_dict, 'Frequency'
)

Interior barrel surface

In [ ]:
plus2_int_aa_pairs = gen_neighbouring_pairs_list(int_barrel_df, 'plus_2', aa_dict_no_cys)
(plus2_int_aa_pair_propensity, plus2_int_aa_pair_frequency, plus2_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_int_aa_pairs, aa_dict_no_cys)

In [ ]:
plus2_int_aa_pair_propensity

In [ ]:
plus2_int_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_int_aa_pair_propensity)

In [ ]:
plot_heat_map(plus2_int_aa_pair_frequency)

In [ ]:
(
    plus2_int_aa_pair_bootstrap_propensity_dict, plus2_int_aa_pair_bootstrap_frequency_dict,
    plus2_int_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, 'plus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_int_aa_pair_propensity, plus2_int_aa_pair_frequency, plus2_int_aa_pair_normed_frequencies
)

In [ ]:
(
    plus2_int_aa_pair_conf_intv_propensity_dict, plus2_int_aa_pair_conf_intv_frequency_dict,
    plus2_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, 'plus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_int_aa_pair_propensity_dict, upper_percentile_plus2_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_int_aa_pair_propensity, lower_percentile_plus2_int_aa_pair_propensity_dict,
    upper_percentile_plus2_int_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_plus2_int_aa_pair_frequency_dict, upper_percentile_plus2_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_int_aa_pair_frequency, lower_percentile_plus2_int_aa_pair_frequency_dict,
    upper_percentile_plus2_int_aa_pair_frequency_dict, 'Frequency'
)

Exterior barrel surface

In [ ]:
plus2_ext_aa_pairs = gen_neighbouring_pairs_list(ext_barrel_df, 'plus_2', aa_dict_no_cys)
(plus2_ext_aa_pair_propensity, plus2_ext_aa_pair_frequency, plus2_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_ext_aa_pairs, aa_dict_no_cys)

In [ ]:
plus2_ext_aa_pair_propensity

In [ ]:
plus2_ext_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_ext_aa_pair_propensity)

In [ ]:
plot_heat_map(plus2_ext_aa_pair_frequency)

In [ ]:
(
    plus2_ext_aa_pair_bootstrap_propensity_dict, plus2_ext_aa_pair_bootstrap_frequency_dict,
    plus2_ext_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, 'plus_2', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_ext_aa_pair_propensity, plus2_ext_aa_pair_frequency, plus2_ext_aa_pair_normed_frequencies
)

In [ ]:
(
    plus2_ext_aa_pair_conf_intv_propensity_dict, plus2_ext_aa_pair_conf_intv_frequency_dict,
    plus2_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, 'plus_2', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_ext_aa_pair_propensity_dict, upper_percentile_plus2_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_ext_aa_pair_propensity, lower_percentile_plus2_ext_aa_pair_propensity_dict,
    upper_percentile_plus2_ext_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_plus2_ext_aa_pair_frequency_dict, upper_percentile_plus2_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_ext_aa_pair_frequency, lower_percentile_plus2_ext_aa_pair_frequency_dict,
    upper_percentile_plus2_ext_aa_pair_frequency_dict, 'Frequency'
)

#### Pairs in Van der Waals contact

Both barrel surfaces

In [ ]:
vdw_aa_pairs = gen_neighbouring_pairs_list(barrel_df, 'van_der_waals', aa_dict_no_cys)
(vdw_aa_pair_propensity, vdw_aa_pair_frequency, vdw_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_aa_pairs, aa_dict_no_cys)

In [ ]:
vdw_aa_pair_propensity

In [ ]:
vdw_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_aa_pair_propensity)

In [ ]:
plot_heat_map(vdw_aa_pair_frequency)

In [ ]:
(
    vdw_aa_pair_bootstrap_propensity_dict, vdw_aa_pair_bootstrap_frequency_dict,
    vdw_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    barrel_df, 'van_der_waals', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_aa_pair_propensity, vdw_aa_pair_frequency, vdw_aa_pair_normed_frequencies
)

In [ ]:
(
    vdw_aa_pair_conf_intv_propensity_dict, vdw_aa_pair_conf_intv_frequency_dict,
    vdw_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    barrel_df, 'van_der_waals', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_aa_pair_propensity_dict, upper_percentile_vdw_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_aa_pair_conf_intv_propensity_dict, 'Propensity'
)

plot_bar_graphs_with_conf_limits(
    vdw_aa_pair_propensity, lower_percentile_vdw_aa_pair_propensity_dict,
    upper_percentile_vdw_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_vdw_aa_pair_frequency_dict, upper_percentile_vdw_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_aa_pair_conf_intv_frequency_dict, 'Frequency'
)

plot_bar_graphs_with_conf_limits(
    vdw_aa_pair_frequency, lower_percentile_vdw_aa_pair_frequency_dict,
    upper_percentile_vdw_aa_pair_frequency_dict, 'Frequency'
)

Interior barrel surface

In [ ]:
vdw_int_aa_pairs = gen_neighbouring_pairs_list(int_barrel_df, 'van_der_waals', aa_dict_no_cys)
(vdw_int_aa_pair_propensity, vdw_int_aa_pair_frequency, vdw_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_int_aa_pairs, aa_dict_no_cys)

In [ ]:
vdw_int_aa_pair_propensity

In [ ]:
vdw_int_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_int_aa_pair_propensity)

In [ ]:
plot_heat_map(vdw_int_aa_pair_frequency)

In [ ]:
(
    vdw_int_aa_pair_bootstrap_propensity_dict, vdw_int_aa_pair_bootstrap_frequency_dict,
    vdw_int_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    int_barrel_df, 'van_der_waals', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_int_aa_pair_propensity, vdw_int_aa_pair_frequency, vdw_int_aa_pair_normed_frequencies
)

In [ ]:
(
    vdw_int_aa_pair_conf_intv_propensity_dict, vdw_int_aa_pair_conf_intv_frequency_dict,
    vdw_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_barrel_df, 'van_der_waals', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_int_aa_pair_propensity_dict, upper_percentile_vdw_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_int_aa_pair_propensity, lower_percentile_vdw_int_aa_pair_propensity_dict,
    upper_percentile_vdw_int_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_vdw_int_aa_pair_frequency_dict, upper_percentile_vdw_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_int_aa_pair_frequency, lower_percentile_vdw_int_aa_pair_frequency_dict,
    upper_percentile_vdw_int_aa_pair_frequency_dict, 'Frequency'
)

Exterior barrel surface

In [ ]:
vdw_ext_aa_pairs = gen_neighbouring_pairs_list(ext_barrel_df, 'van_der_waals', aa_dict_no_cys)
(vdw_ext_aa_pair_propensity, vdw_ext_aa_pair_frequency, vdw_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_ext_aa_pairs, aa_dict_no_cys)

In [ ]:
vdw_ext_aa_pair_propensity

In [ ]:
vdw_ext_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_ext_aa_pair_propensity)

In [ ]:
plot_heat_map(vdw_ext_aa_pair_frequency)

In [ ]:
(
    vdw_ext_aa_pair_bootstrap_propensity_dict, vdw_ext_aa_pair_bootstrap_frequency_dict,
    vdw_ext_aa_pair_bootstrap_normed_frequencies_dict
) = iterate_bootstrap_propensities(
    ext_barrel_df, 'van_der_waals', aa_dict_no_cys, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_ext_aa_pair_propensity, vdw_ext_aa_pair_frequency, vdw_ext_aa_pair_normed_frequencies
)

In [ ]:
(
    vdw_ext_aa_pair_conf_intv_propensity_dict, vdw_ext_aa_pair_conf_intv_frequency_dict,
    vdw_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_barrel_df, 'van_der_waals', aa_dict_no_cys, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_ext_aa_pair_propensity_dict, upper_percentile_vdw_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_ext_aa_pair_propensity, lower_percentile_vdw_ext_aa_pair_propensity_dict,
    upper_percentile_vdw_ext_aa_pair_propensity_dict, 'Propensity'
)

In [ ]:
(
    lower_percentile_vdw_ext_aa_pair_frequency_dict, upper_percentile_vdw_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_ext_aa_pair_frequency, lower_percentile_vdw_ext_aa_pair_frequency_dict,
    upper_percentile_vdw_ext_aa_pair_frequency_dict, 'Frequency'
)

## Continuous propensity calculations

### 1D individual amino acid propensities

Both barrel surfaces

In [ ]:
plot_aa_kdes(barrel_df, ['z_coords'], 'comparison', aa_dict_no_cys)

In [ ]:
z_propensity = plot_1d_indv_aa_propensities(barrel_df, 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
plot_aa_kdes(int_barrel_df, ['z_coords'], 'comparison', aa_dict_no_cys)

In [ ]:
z_int_propensity = plot_1d_indv_aa_propensities(int_barrel_df, 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
plot_aa_kdes(ext_barrel_df, ['z_coords'], 'comparison', aa_dict_no_cys)

In [ ]:
z_ext_propensity = plot_1d_indv_aa_propensities(ext_barrel_df, 'z_coords', aa_dict_no_cys)

### 1D pairwise amino acid propensities

#### Hydrogen bonding pairs

Both barrel surfaces

In [ ]:
z_hb_propensity = plot_1d_pairwise_aa_propensities(barrel_df, 'hb_pairs', 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
z_hb_int_propensity = plot_1d_pairwise_aa_propensities(int_barrel_df, 'hb_pairs', 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
z_hb_ext_propensity = plot_1d_pairwise_aa_propensities(ext_barrel_df, 'hb_pairs', 'z_coords', aa_dict_no_cys)

#### Non- hydrogen bonding pairs

Both barrel surfaces

In [ ]:
z_nhb_propensity = plot_1d_pairwise_aa_propensities(barrel_df, 'nhb_pairs', 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
z_nhb_int_propensity = plot_1d_pairwise_aa_propensities(int_barrel_df, 'nhb_pairs', 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
z_nhb_ext_propensity = plot_1d_pairwise_aa_propensities(ext_barrel_df, 'nhb_pairs', 'z_coords', aa_dict_no_cys)

#### Minus 2 pairs

Both barrel surfaces

In [ ]:
z_minus2_propensity = plot_1d_pairwise_aa_propensities(barrel_df, 'minus_2', 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
z_minus2_int_propensity = plot_1d_pairwise_aa_propensities(int_barrel_df, 'minus_2', 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
z_minus2_ext_propensity = plot_1d_pairwise_aa_propensities(ext_barrel_df, 'minus_2', 'z_coords', aa_dict_no_cys)

#### Plus 2 pairs

Both barrel surfaces

In [ ]:
z_plus2_propensity = plot_1d_pairwise_aa_propensities(barrel_df, 'plus_2', 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
z_plus2_int_propensity = plot_1d_pairwise_aa_propensities(int_barrel_df, 'plus_2', 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
z_plus2_ext_propensity = plot_1d_pairwise_aa_propensities(ext_barrel_df, 'plus_2', 'z_coords', aa_dict_no_cys)

#### Van der Waals pairs

Both barrel surfaces

In [ ]:
z_vdw_propensity = plot_1d_pairwise_aa_propensities(barrel_df, 'van_der_waals', 'z_coords', aa_dict_no_cys)

Interior barrel surface

In [ ]:
z_vdw_int_propensity = plot_1d_pairwise_aa_propensities(int_barrel_df, 'van_der_waals', 'z_coords', aa_dict_no_cys)

Exterior barrel surface

In [ ]:
z_vdw_ext_propensity = plot_1d_pairwise_aa_propensities(ext_barrel_df, 'van_der_waals', 'z_coords', aa_dict_no_cys)

### 2D individual amino acid propensities

Both barrel surfaces

In [ ]:
cont_phi_psi_propensity = plot_2d_indv_aa_propensities(barrel_df, 'phi', 'psi', aa_dict_no_cys,
                                                       [-180, -60], [100, 180])

Interior barrel surface

In [ ]:
cont_phi_psi_int_propensity = plot_2d_indv_aa_propensities(int_barrel_df, 'phi', 'psi', aa_dict_no_cys,
                                                           [-180, -60], [100, 180])

Exterior barrel surface

In [ ]:
cont_phi_psi_ext_propensity = plot_2d_indv_aa_propensities(ext_barrel_df, 'phi', 'psi', aa_dict_no_cys,
                                                           [-180, -60], [100, 180])

## Saves propensity and frequency dictionaries to pickle file

In [ ]:
propensities_dict = OrderedDict({'int_-_-_-_hb_pair_disc_propensity': hb_int_aa_pair_propensity,
                                 'ext_-_-_-_hb_pair_disc_propensity': hb_ext_aa_pair_propensity,
                                 'int_-_-_-_nhb_pair_disc_propensity': nhb_int_aa_pair_propensity,
                                 'ext_-_-_-_nhb_pair_disc_propensity': nhb_ext_aa_pair_propensity,
                                 'int_-_-_-_plusminus2_pair_disc_propensity': minus2_int_aa_pair_propensity,
                                 'ext_-_-_-_plusminus2_pair_disc_propensity': minus2_ext_aa_pair_propensity,
                                 'int_-_-_-_vdw_pair_disc_propensity': vdw_int_aa_pair_propensity,
                                 'ext_-_-_-_vdw_pair_disc_propensity': vdw_ext_aa_pair_propensity,
                                 'int_-_z_-_-_indv_cont_propensity': z_int_propensity,
                                 'ext_-_z_-_-_indv_cont_propensity': z_ext_propensity,
                                 'int_-_z_-_hb_pair_cont_propensity': z_hb_int_propensity,
                                 'ext_-_z_-_hb_pair_cont_propensity': z_hb_ext_propensity,
                                 'int_-_z_-_nhb_pair_cont_propensity': z_nhb_int_propensity,
                                 'ext_-_z_-_nhb_pair_cont_propensity': z_nhb_ext_propensity,
                                 'int_-_z_-_plusminus2_pair_cont_propensity': z_minus2_int_propensity,
                                 'ext_-_z_-_plusminus2_pair_cont_propensity': z_minus2_ext_propensity,
                                 'int_-_z_-_vdw_pair_cont_propensity': z_vdw_int_propensity,
                                 'ext_-_z_-_vdw_pair_cont_propensity': z_vdw_ext_propensity,
                                 'int_-_phi_psi_-_indv_disc_propensity': discrete_phi_psi_int_propensity,
                                 'ext_-_phi_psi_-_indv_disc_propensity': discrete_phi_psi_ext_propensity})

with open('Pickled_propensity_dictionaries.pkl', 'wb') as pickle_file:
    pickle.dump(propensities_dict, pickle_file)

In [ ]:
frequencies_dict = OrderedDict({'int_-_-_-_hb_pair_disc_frequency': hb_int_aa_pair_normed_frequencies,
                                'ext_-_-_-_hb_pair_disc_frequency': hb_ext_aa_pair_normed_frequencies,
                                'int_-_-_-_nhb_pair_disc_frequency': nhb_int_aa_pair_normed_frequencies,
                                'ext_-_-_-_nhb_pair_disc_frequency': nhb_ext_aa_pair_normed_frequencies,
                                'int_-_-_-_plusminus2_pair_disc_frequency': minus2_int_aa_pair_normed_frequencies,
                                'ext_-_-_-_plusminus2_pair_disc_frequency': minus2_ext_aa_pair_normed_frequencies,
                                'int_-_-_-_vdw_pair_disc_frequency': vdw_int_aa_pair_normed_frequencies,
                                'ext_-_-_-_vdw_pair_disc_frequency': vdw_ext_aa_pair_normed_frequencies,
                                'int_-_phi_psi_-_indv_disc_frequency': discrete_phi_psi_int_normed_frequencies,
                                'ext_-_phi_psi_-_indv_disc_frequency': discrete_phi_psi_ext_normed_frequencies})

with open('Pickled_frequency_dictionaries.pkl', 'wb') as pickle_file:
    pickle.dump(frequencies_dict, pickle_file)

## 2D Kolmogorov-Smirnov test to compare individual and overall amino acid z-coordinate distributions

Both barrel surfaces

How many bootstrap samples should I use (all residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    barrel_df, 'z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000, 30000, 100000], aa_dict_no_cys
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict, initial_p_dict = initial_ks_test(barrel_df, 'z_coords', run_ks_2samp_test, aa_dict_no_cys)

D_dict_bootstrap, p_dict_bootstrap = bootstrap_ks_test(barrel_df, 'z_coords', 10000, run_ks_2samp_test, aa_dict_no_cys)
draw_plot(sns.violinplot, p_dict_bootstrap, 'Amino acid', 'p')

barrel_conf_intv_p_dict = calc_95_conf_limits(initial_p_dict, p_dict_bootstrap)
draw_plot(sns.swarmplot, barrel_conf_intv_p_dict, 'Amino acid', 'p')

Interior surface barrels

How many bootstrap samples should I use (interior residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    int_barrel_df, 'z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000, 30000, 100000], aa_dict_no_cys
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict_int, initial_p_dict_int = initial_ks_test(int_barrel_df, 'z_coords', run_ks_2samp_test, aa_dict_no_cys)

D_dict_bootstrap_int, p_dict_bootstrap_int = bootstrap_ks_test(
    int_barrel_df, 'z_coords', 10000, run_ks_2samp_test, aa_dict_no_cys
)
draw_plot(sns.violinplot, p_dict_bootstrap_int, 'Amino acid', 'p')

barrel_conf_intv_p_dict_int = calc_95_conf_limits(initial_p_dict_int, p_dict_bootstrap_int)
draw_plot(sns.swarmplot, barrel_conf_intv_p_dict_int, 'Amino acid', 'p')

Exterior barrel surface

How many bootstrap samples should I use (exterior residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    ext_barrel_df, 'z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000, 30000, 100000], aa_dict_no_cys
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict_ext, initial_p_dict_ext = initial_ks_test(ext_barrel_df, 'z_coords', run_ks_2samp_test, aa_dict_no_cys)

D_dict_bootstrap_ext, p_dict_bootstrap_ext = bootstrap_ks_test(
    ext_barrel_df, 'z_coords', 10000, run_ks_2samp_test, aa_dict_no_cys
)
draw_plot(sns.violinplot, p_dict_bootstrap_ext, 'Amino acid', 'p')

barrel_conf_intv_p_dict_ext = calc_95_conf_limits(initial_p_dict_ext, p_dict_bootstrap_ext)
draw_plot(sns.swarmplot, barrel_conf_intv_p_dict_ext, 'Amino acid', 'p')

## Interaction type vs. z-coordinate KDEs

In [ ]:
plot_discrete_prop_frequency_kdes(
    barrel_df, 'z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking', 'pi_pi_stacking_p',
                            'pi_pi_stacking_l', 'pi_pi_stacking_n', 'pi_pi_stacking_t', 'cation_pi']
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    int_barrel_df, 'z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking', 'pi_pi_stacking_p',
                                'pi_pi_stacking_l', 'pi_pi_stacking_n', 'pi_pi_stacking_t', 'cation_pi']
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    ext_barrel_df, 'z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking', 'pi_pi_stacking_p',
                                'pi_pi_stacking_l', 'pi_pi_stacking_n', 'pi_pi_stacking_t', 'cation_pi']
)

## Standard error of proportion

Recalculate HB / NHB standard error of proportion z-scores from count data in Dek's 1998 paper

In [ ]:
dek_sandwich_hb_pairs_dict, dek_sandwich_hb_pairs_total = gen_dek_sandwich_dicts(aa_dict, 'hb')
dek_sandwich_nhb_pairs_dict, dek_sandwich_nhb_pairs_total = gen_dek_sandwich_dicts(aa_dict, 'nhb')

dek_sandwich_hb_nhb_z_scores, dek_sandwich_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict, dek_sandwich_hb_pairs_dict, dek_sandwich_nhb_pairs_dict, dek_sandwich_hb_pairs_total,
    dek_sandwich_nhb_pairs_total
)

In [ ]:
dek_sandwich_hb_nhb_ratios

In [ ]:
dek_sandwich_hb_nhb_z_scores

In [ ]:
print_significant_z_scores(dek_sandwich_hb_nhb_z_scores)

Calculate HB / NHB standard error of proportion z-scores for new data

In [ ]:
barrel_df_hb_pairs_dict, barrel_df_hb_pairs_total = calc_pair_proportions(
    barrel_df, 'hb_pairs', aa_dict_no_cys
)
barrel_df_nhb_pairs_dict, barrel_df_nhb_pairs_total = calc_pair_proportions(
    barrel_df, 'nhb_pairs', aa_dict_no_cys
)

barrel_df_hb_nhb_z_scores, barrel_df_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict_no_cys, barrel_df_hb_pairs_dict, barrel_df_nhb_pairs_dict, barrel_df_hb_pairs_total,
    barrel_df_nhb_pairs_total
)

In [ ]:
barrel_df_hb_nhb_ratios

In [ ]:
barrel_df_hb_nhb_z_scores

In [ ]:
print_significant_z_scores(barrel_df_hb_nhb_z_scores)

In [ ]:
barrel_df_dek_sandwich_z_diff_dict = calc_z_diff(barrel_df_hb_nhb_z_scores, dek_sandwich_hb_nhb_z_scores)
barrel_df_dek_sandwich_z_diff_dict

Interior barrel surface (new dataset only)

In [ ]:
int_barrel_df_hb_pairs_dict, int_barrel_df_hb_pairs_total = calc_pair_proportions(
    int_barrel_df, 'hb_pairs', aa_dict_no_cys
)
int_barrel_df_nhb_pairs_dict, int_barrel_df_nhb_pairs_total = calc_pair_proportions(
    int_barrel_df, 'nhb_pairs', aa_dict_no_cys
)

int_barrel_df_hb_nhb_z_scores, int_barrel_df_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict_no_cys, int_barrel_df_hb_pairs_dict, int_barrel_df_nhb_pairs_dict, int_barrel_df_hb_pairs_total,
    int_barrel_df_nhb_pairs_total
)

In [ ]:
int_barrel_df_hb_nhb_ratios

In [ ]:
int_barrel_df_hb_nhb_z_scores

In [ ]:
print_significant_z_scores(int_barrel_df_hb_nhb_z_scores)

Exterior barrel surface (new dataset only)

In [ ]:
ext_barrel_df_hb_pairs_dict, ext_barrel_df_hb_pairs_total = calc_pair_proportions(
    ext_barrel_df, 'hb_pairs', aa_dict_no_cys
)
ext_barrel_df_nhb_pairs_dict, ext_barrel_df_nhb_pairs_total = calc_pair_proportions(
    ext_barrel_df, 'nhb_pairs', aa_dict_no_cys
)

ext_barrel_df_hb_nhb_z_scores, ext_barrel_df_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict_no_cys, ext_barrel_df_hb_pairs_dict, ext_barrel_df_nhb_pairs_dict, ext_barrel_df_hb_pairs_total,
    ext_barrel_df_nhb_pairs_total
)

In [ ]:
ext_barrel_df_hb_nhb_ratios

In [ ]:
ext_barrel_df_hb_nhb_z_scores

In [ ]:
print_significant_z_scores(ext_barrel_df_hb_nhb_z_scores)

## Additional functions

In [ ]:
def calc_discrete_2d_pairwise_aa_propensities(df, cont_prop_1, cont_prop_2, iteraction, aa_dict, cluster_coords):
    """
    Calculates pairwise amino acid propensity values for bins in 2D property space.
    NOTE: This function calculates the propensity for an amino acid pair to be found in a particular bin (as
    compared with all other bins), rather than the propensity for one amino acid to interact with a second within
    that bin.
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the corresponding column in the input dataframe), second (continuous) property of interest
           (specified via the name of the corresponding column in the input dataframe), type of interaction between
           the amino acids in each pair, dictionary of amino acid abbreviations, and coordinates of cluster centres
    Returns: dataframe of propensity values, plus dataframes of frequency and normalised frequency values (since
             propensity values can be skewed by very small sample sizes)
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    neighbouring_pairs_list, props_dict = gen_neighbouring_pairs_list(
        df, interaction, aa_dict, cont_prop_1, cont_prop_2
    )
    prop_1_dict = props_dict[cont_prop_1]
    prop_2_dict = props_dict[cont_prop_2]

    aa_1_aa_2_list = [''.join(aa_pair) for aa_pair in list(itertools.product(list(aa_dict.keys())))]
    propensity_dict = OrderedDict({'FASTA': aa_1_aa_2_list})
    frequency_dict = OrderedDict({'FASTA': aa_1_aa_2_list})
    normed_frequencies_dict = OrderedDict({'FASTA': aa_1_aa_2_list})
    voronoi_class_dict = OrderedDict()

    for voronoi_index in range(cluster_coords.shape[0]):
        voronoi_class_dict[voronoi_index] = []
        propensity_dict[voronoi_index] = [np.nan]*len(aa_1_aa_2_list)
        frequency_dict[voronoi_index] = [np.nan]*len(aa_1_aa_2_list)
        normed_frequencies_dict[voronoi_index] = [np.nan]*len(aa_1_aa_2_list)

    for pair in list(prop_1_dict.keys()):
        prop_val_1 = prop_1_dict[pair][0]
        prop_val_2 = prop_2_dict[pair][0]
        prop_vals = np.array([prop_val_1, prop_val_2])
        
        distances = np.sqrt(np.sum(np.square(cluster_coords-prop_vals), axis=1))
        voronoi_index = np.abs(distances).argmin()
        voronoi_class_dict[voronoi_index].append(''.join(pair.split('_')[2:]))

    for aa_1 in list(aa_dict.keys()):
        for aa_2 in list(aa_dict.keys()):
            aa_1_aa_2_index = aa_1_aa_2_list.index('{}{}'.format(aa_1, aa_2))

            for voronoi_index in list(voronoi_class_dict.keys()):
                class_aa_1_aa_2_count = voronoi_class_dict[voronoi_index].count('{}{}'.format(aa_1, aa_2))
                class_all_aas_count = len(voronoi_class_dict[voronoi_index])
                total_aa_1_aa_2_count = neighbouring_pairs_list.count('{}{}'.format(aa_1, aa_2))
                total_count = len(neighbouring_pairs_list)

                try:
                    propensity_dict[voronoi_index][aa_1_aa_2_index] = (  (class_aa_1_aa_2_count / class_all_aas_count)
                                                                       / (total_aa_1_aa_2_count / total_count))
                    frequency_dict[voronoi_index][aa_1_aa_2_index] = copy.deepcopy(class_aa_1_aa_2_count)
                    normed_frequencies_dict[voronoi_index][aa_1_aa_2_index] = class_aa_1_aa_2_count / class_all_aas_count
                except ZeroDivisionError:
                    pass

    propensity_df = pd.DataFrame(propensity_dict)
    frequency_df = pd.DataFrame(frequency_dict)
    normed_frequencies_df = pd.DataFrame(normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df


def calc_discrete_binned_2d_pairwise_aa_propensities(df, cont_prop_1, cont_prop_2, iteraction, aa_dict,
                                                     cluster_coords):
    """
    Calculates pairwise amino acid propensity values for bins in 2D property space.
    NOTE: This function calculates the propensity for one amino acid to interact with a second within a particular
    bin, rather than the propensity of an amino acid to be found within that bin as compared to all others.
    **Consequently, this calculation requires a large number of data points in each bin.**
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the corresponding column in the input dataframe), second (continuous) property of interest
           (specified via the name of the corresponding column in the input dataframe), type of interaction between
           the amino acids in each pair, dictionary of amino acid abbreviations, and coordinates of cluster centres
    Returns: dataframe of propensity values, plus dataframes of frequency and normalised frequency values (since
             propensity values can be skewed by very small sample sizes)
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    neighbouring_pairs_list, props_dict = gen_neighbouring_pairs_list(
        df, interaction, aa_dict, cont_prop_1, cont_prop_2
    )
    prop_1_dict = props_dict[cont_prop_1]
    prop_2_dict = props_dict[cont_prop_2]

    propensity_dict = OrderedDict()
    frequency_dict = OrderedDict()
    normed_frequencies_dict = OrderedDict()
    voronoi_class_dict = OrderedDict()

    for voronoi_index in range(cluster_coords.shape[0]):
        voronoi_class_dict[voronoi_index] = []
        propensity_dict[voronoi_index] = OrderedDict({'FASTA': list(aa_dict.keys())})
        frequency_dict[voronoi_index] = OrderedDict({'FASTA': list(aa_dict.keys())})
        normed_frequencies_dict[voronoi_index] = OrderedDict({'FASTA': list(aa_dict.keys())})

    for pair in list(prop_1_dict.keys()):
        prop_val_1 = prop_1_dict[pair][0]
        prop_val_2 = prop_2_dict[pair][0]
        prop_vals = np.array([prop_val_1, prop_val_2])

        distances = np.sqrt(np.sum(np.square(cluster_coords-prop_vals), axis=1))
        voronoi_index = np.abs(distances).argmin()
        voronoi_class_dict[voronoi_index].append(''.join(pair.split('_')[2:]))

    for voronoi_index in list(voronoi_class_dict.keys()):
        for aa_1 in list(aa_dict.keys()):
            propensity_dict[voronoi_index][aa_1] = [np.nan]*len(aa_dict)
            frequency_dict[voronoi_index][aa_1] = [np.nan]*len(aa_dict)
            normed_frequencies_dict[voronoi_index][aa_1] = [np.nan]*len(aa_dict)

            for aa_2_index, aa_2 in list(aa_dict.keys()):
                aa_1_aa_2_count = 0
                aa_1_count = 0
                aa_2_count = 0
                all_aas_count = 0
                for pair in voronoi_class_dict[voronoi_index]:
                    all_aas_count += 1
                    if pair[0] == aa_1:
                        aa_1_count += 1
                    if pair[1] == [aa_2]:
                        aa_2_count += 1
                    if pair[0] == aa_1 and pair[1] == aa_2:
                        aa_1_aa_2_count += 1

                try:
                    frequency_dict[voronoi_index][aa_1][aa_2_index] = copy.deepcopy(aa_1_aa_2_count)
                    normed_frequencies_dict[voronoi_index][aa_1][aa_2_index] = aa_1_aa_2_count / aa_1_count
                    propensity_dict[voronoi_index][aa_1][aa_2_index] = (  (aa_1_aa_2_count / aa_1_count)
                                                                        / (aa_2_count / all_aas_count))
                except ZeroDivisionError:
                    pass

    for voronoi_index in list(propensity_dict.keys()):
        propensity_dict[voronoi_index] = pd.DataFrame(propensity_dict[voronoi_index])
        frequency_dict[voronoi_index] = pd.DataFrame(frequency_dict[voronoi_index])
        normed_frequencies_dict[voronoi_index] = pd.DataFrame(normed_frequencies_dict[voronoi_index])

    return propensity_dict, frequency_dict, normed_frequencies_dict